# 神经网络快速入门与实践

## 基于NumPy的神经网络构建与训练

### 示例1：最简单的神经网络模型（y=wx+b）
- 单个神经元构成的单层网络（仅有一个输出层）
- 输入特征为1
- 输入样本数量为1

In [ ]:
import numpy as np

# 数据
x = 2.0   # 输入，
y = 4.0   # 真实值

# 参数初始化
w = 1.0
b = 0.0
lr = 0.1

# 前向传播
y_hat = w * x + b
loss = 0.5 * (y_hat - y) ** 2

# 反向传播
dL_dyhat = y_hat - y
dL_dw = dL_dyhat * x
dL_db = dL_dyhat * 1

# 参数更新
w = w - lr * dL_dw
b = b - lr * dL_db

print(f"Updated w: {w}, b: {b}")

### 示例2：单层网络，输入为矩阵形式（y=Xw+b）
- 扩展前面的示例：
  - 输入特征为2
  - 样本数量为2，每个样本2个特征

In [ ]:
import numpy as np

# 模拟数据
X = np.array([[1.0, 2.0], [3.0, 4.0]])  # shape: (2, 2)，输入特征为2，样本数量为2
y = np.array([[5.0], [11.0]])          # shape: (2, 1)，每个样本都有一个真实值

# 初始化参数
# 两个输入特征的各初始权重，因为只有一个神经元，
# 这里的权重矩阵表现为列向量的形式
b = 0.0
w = np.array([[0.1], [0.1]])           # shape: (2, 1)
lr = 0.01
N = X.shape[0]

# 前向传播
y_hat = X @ w + b
loss = 0.5 * np.mean((y_hat - y) ** 2)

# 反向传播
grad_yhat = (y_hat - y) / N
grad_w = X.T @ grad_yhat
grad_b = np.sum(grad_yhat)

# 更新参数
w -= lr * grad_w
b -= lr * grad_b

print("Updated w:", w.ravel())
print("Updated b:", b)

### 示例3：两层网络示例（y=XW+b）
- 两层网络：一个隐藏层，一个输出层
- 样本数为N=2，输入特征维度为D=2
- 隐藏层神经元数H=4，使用ReLU激活函数
- 损失函数：均方误差（MSE）
  - $L = \frac{1}{2N} \sum_{i=1}^N (\hat{y}_i - y_i)^2$

**神经网络结构：**
```mathematica
输入 X ∈ ℝ^{N×D}
   ↓
第一层（线性）: Z₁ = X @ W₁ + b₁      # shape: (N, H)
   ↓
ReLU 激活函数: A₁ = ReLU(Z₁)           # shape: (N, H)
   ↓
第二层（线性）: ŷ = A₁ @ W₂ + b₂      # shape: (N, 1)
   ↓
MSE 损失函数: L = MSE(ŷ, y)
```

其中的符号表示：
- 𝐷：输入特征数
- 𝐻：隐藏层维度（神经元数）
- 𝑁：样本数
- ReLU激活：ReLU(𝑧)=max(0,𝑧)

**前向传播与反向传播流程图**
```scss
前向传播：
X → Linear(W1, b1) → Z1 → ReLU → A1 → Linear(W2, b2) → y_hat → MSE → Loss

反向传播（链式）：
Loss → d(y_hat) → dW2, db2 → dA1 → dZ1 (ReLU) → dW1, db1
```

#### 代码解析
##### 1. 前向传播
首先明确前向传播的计算过程，以便后续推导梯度。代码中的前向传播步骤如下：

（1）隐藏层计算：$Z_1 = X W_1 + b_1$
- $ X \in \mathbb{R}^{N \times D} $，$ W_1 \in \mathbb{R}^{D \times H} $，$ b_1 \in \mathbb{R}^{1 \times H} $。
- $ Z_1 \in \mathbb{R}^{N \times H} $ 是隐藏层的线性输出。
- 代码实现
  ```python
  Z1 = X @ W1 + b1  # (N x H)
  ```
（2）ReLU激活：$A_1 = \text{ReLU}(Z_1) = \max(0, Z_1)$
- ReLU 逐元素操作：如果  $ Z_1_{ij} > 0 $，则 A_1_{ij} = Z_1_{ij} ，否则 A_1_{ij} = 0 。
- $ A_1 \in \mathbb{R}^{N \times H} $。
- 代码实现
  ```python
  A1 = np.maximum(0, Z1)  # (N x H)
  ```

（3）输出层计算：$\hat{y} = A_1 W_2 + b_2$
- $ A_1 \in \mathbb{R}^{N \times H} $，$ W_2 \in \mathbb{R}^{H \times 1} $，$ b_2 \in \mathbb{R}^{1 \times 1} $。
- $ \hat{y} \in \mathbb{R}^{N \times 1} $。
- 代码实现
  ```python
  y_hat = A1 @ W2 + b2  # (N x 1)
  ```

（4）损失函数：$L = \frac{1}{2N} \sum_{i=1}^N (\hat{y}_i - y_i)^2$
- 代码计算均方误差的平均值
  ```python
  loss = 0.5 * np.mean((y_hat - y) ** 2)
  ```

##### 2. 反向传播
反向传播的目标是计算损失 $ L $ 对参数 $ W_1, b_1, W_2, b_2 $ 的梯度。我们使用链式法则，从输出层向输入层逐步推导。

（1）**对损失函数的导数**

对 $ \hat{y} $ 求导：$\frac{\partial L}{\partial \hat{y}_i} = \frac{1}{N} (\hat{y}_i - y_i)$

矩阵形式：$\frac{\partial L}{\partial \hat{y}} = \frac{1}{N} (\hat{y} - y) \in \mathbb{R}^{N \times 1}$

代码实现：
```python
dy_hat = (y_hat - y) / N  # (N x 1)
```

（2）**输出层参数梯度 ($ W_2, b_2 $)**

输出层计算为：$\hat{y} = A_1 W_2 + b_2$

梯度 $ \frac{\partial L}{\partial W_2} $
- 根据链式法则：$\frac{\partial L}{\partial W_2} = \frac{\partial L}{\partial \hat{y}} \cdot \frac{\partial \hat{y}}{\partial W_2}$
- 其中：$\frac{\partial \hat{y}}{\partial W_2} = A_1$

因此：$\frac{\partial L}{\partial W_2} = A_1^T \cdot \frac{\partial L}{\partial \hat{y}}$
- $ A_1^T \in \mathbb{R}^{H \times N} $，$ \frac{\partial L}{\partial \hat{y}} \in \mathbb{R}^{N \times 1} $。
- 结果：$ \frac{\partial L}{\partial W_2} \in \mathbb{R}^{H \times 1} $。

代码实现：
```python
dW2 = A1.T @ dy_hat  # (H x 1)
```

梯度$ \frac{\partial L}{\partial b_2} $：
- 根据链式法则：$\frac{\partial L}{\partial b_2} = \frac{\partial L}{\partial \hat{y}} \cdot \frac{\partial \hat{y}}{\partial b_2}$
- 其中：$\frac{\partial \hat{y}}{\partial b_2} = 1$

因此：$\frac{\partial L}{\partial b_2} = \text{sum}(\frac{\partial L}{\partial \hat{y}}, \text{axis}=0)$
- 对 $ \frac{\partial L}{\partial \hat{y}} \in \mathbb{R}^{N \times 1} $ 按行求和，保持维度为 $ (1 \times 1) $。

代码实现：
```python
db2 = np.sum(dy_hat, axis=0, keepdims=True)  # (1 x 1)
```

（3）**传播到隐藏层**

需要计算 $ \frac{\partial L}{\partial A_1} $ 和 $ \frac{\partial L}{\partial Z_1} $，以便进一步推导 $ W_1, b_1 $ 的梯度。

梯度 $ \frac{\partial L}{\partial A_1} $：
- 根据链式法则：$\frac{\partial L}{\partial A_1} = \frac{\partial L}{\partial \hat{y}} \cdot \frac{\partial \hat{y}}{\partial A_1}$
- 其中：$\hat{y} = A_1 W_2 + b_2$， $\frac{\partial \hat{y}}{\partial A_1} = W_2$

因此：$\frac{\partial L}{\partial A_1} = \frac{\partial L}{\partial \hat{y}} \cdot W_2^T$
- $ \frac{\partial L}{\partial \hat{y}} \in \mathbb{R}^{N \times 1} $，$ W_2^T \in \mathbb{R}^{1 \times H} $。
- 结果：$ \frac{\partial L}{\partial A_1} \in \mathbb{R}^{N \times H} $。

代码实现：
```python
dA1 = dy_hat @ W2.T  # (N x H)
```

梯度 $ \frac{\partial L}{\partial Z_1} $：
- 隐藏层使用ReLU激活函数，$A_1 = \text{ReLU}(Z_1) = \max(0, Z_1)$
- ReLU的导数为：$\frac{\partial A_1}{\partial Z_1} = \begin{cases} 
1, & \text{if } Z_1 > 0 \\
0, & \text{if } Z_1 \leq 0 
\end{cases}$

因此：$\frac{\partial L}{\partial Z_1} = \frac{\partial L}{\partial A_1} \cdot \frac{\partial A_1}{\partial Z_1}$
- $ \frac{\partial A_1}{\partial Z_1} $ 是一个逐元素操作，值为 1（当 $ Z_1 > 0 $）或 0（当 $ Z_1 \leq 0 $）。
- 代码中用 (Z1 > 0).astype(float) 实现ReLU的导数。
- 结果：$ \frac{\partial L}{\partial Z_1} \in \mathbb{R}^{N \times H} $。

代码实现：
```python
dZ1 = dA1 * (Z1 > 0).astype(float)  # (N x H)
```

（4）**隐藏层参数梯度 ($ W_1, b_1 $)**

隐藏层计算为：$Z_1 = X W_1 + b_1$

梯度 $ \frac{\partial L}{\partial W_1} $
- 根据链式法则：$\frac{\partial L}{\partial W_1} = \frac{\partial L}{\partial Z_1} \cdot \frac{\partial Z_1}{\partial W_1}$
- 其中：$\frac{\partial Z_1}{\partial W_1} = X$

因此：$\frac{\partial L}{\partial W_1} = X^T \cdot \frac{\partial L}{\partial Z_1}$
- $ X^T \in \mathbb{R}^{D \times N} $，$ \frac{\partial L}{\partial Z_1} \in \mathbb{R}^{N \times H} $。
- 结果：$ \frac{\partial L}{\partial W_1} \in \mathbb{R}^{D \times H} $。

代码实现：
```python
dW1 = X.T @ dZ1  # (D x H)
```

梯度 $ \frac{\partial L}{\partial b_1} $：
- 根据链式法则：$\frac{\partial L}{\partial b_1} = \frac{\partial L}{\partial Z_1} \cdot \frac{\partial Z_1}{\partial b_1}$
- 其中：$\frac{\partial Z_1}{\partial b_1} = 1$

因此：$\frac{\partial L}{\partial b_1} = \sum_{i=1}^N \frac{\partial L}{\partial Z_{1,i}}$
- 对 $ \frac{\partial L}{\partial Z_1} \in \mathbb{R}^{N \times H} $ 按行求和，保持维度为 $ (1 \times H) $。

代码实现：
```python
db1 = np.sum(dZ1, axis=0, keepdims=True)  # (1 x H)
```

##### 3. 参数更新
使用梯度下降更新参数：$\theta \gets \theta - \eta \cdot \frac{\partial L}{\partial \theta}$

其中 $ \eta = 0.01 $ 是学习率，$ \theta $ 代表 $ W_1, b_1, W_2, b_2 $。






In [ ]:
import numpy as np

# ---------- 初始化 ----------
np.random.seed(42)

# 输入数据 X 和目标 y
X = np.array([[1.0, 2.0], [3.0, 4.0]])  # shape: (2, 2)
y = np.array([[5.0], [11.0]])          # shape: (2, 1)
N, D = X.shape
H = 4  # 隐藏层单元数
lr = 0.01

# 权重初始化
W1 = np.random.randn(D, H) * 0.01     # 第一层权重 (D, H)
b1 = np.zeros((1, H))                 # 第一层偏置 (1, H)
W2 = np.random.randn(H, 1) * 0.01     # 第二层权重 (H, 1)
b2 = np.zeros((1, 1)) 

# ---------- 前向传播 ----------
Z1 = X @ W1 + b1            # (N x H)
A1 = np.maximum(0, Z1)      # ReLU 激活 → (N x H)
y_hat = A1 @ W2 + b2        # 输出 → (N x 1)
loss = 0.5 * np.mean((y_hat - y) ** 2)

# ---------- 反向传播 ----------
dy_hat = (y_hat - y) / N                # dL/dy_hat → (N x 1)

# 输出层参数梯度
dW2 = A1.T @ dy_hat                     # (H x 1)
db2 = np.sum(dy_hat, axis=0, keepdims=True)  # (1 x 1)

# 传播到隐藏层
dA1 = dy_hat @ W2.T                     # (N x H)
dZ1 = dA1 * (Z1 > 0).astype(float)      # ReLU 导数 → (N x H)

# 隐藏层参数梯度
dW1 = X.T @ dZ1                         # (D x H)
db1 = np.sum(dZ1, axis=0, keepdims=True)  # (1 x H)

# ---------- 参数更新 ----------
W1 -= lr * dW1
b1 -= lr * db1
W2 -= lr * dW2
b2 -= lr * db2

# ---------- 输出 ----------
print("Loss:", loss)
print("Updated W1:\n", W1)
print("Updated b1:\n", b1)
print("Updated W2:\n", W2)
print("Updated b2:\n", b2)

### 扩展以上网络，使用两个隐藏层的示例

In [ ]:
import numpy as np

# 设置随机种子
np.random.seed(42)

# ---------- 模拟数据 ----------
X = np.array([[1.0, 2.0], [3.0, 4.0]])  # shape: (2, 2)
y = np.array([[5.0], [11.0]])          # shape: (2, 1)
N, D = X.shape

# 网络结构参数
H1 = 4  # 第 1 个隐藏层神经元个数
H2 = 4  # 第 2 个隐藏层神经元个数
lr = 0.01

# ---------- 初始化参数 ----------
W1 = np.random.randn(D, H1) * np.sqrt(2. / D)
b1 = np.zeros((1, H1))

W2 = np.random.randn(H1, H2) * np.sqrt(2. / H1)
b2 = np.zeros((1, H2))

W3 = np.random.randn(H2, 1) * np.sqrt(2. / H2)
b3 = np.zeros((1, 1))

# ---------- 前向传播 ----------
Z1 = X @ W1 + b1         # (N, H1)
A1 = np.maximum(0, Z1)   # ReLU → (N, H1)

Z2 = A1 @ W2 + b2        # (N, H2)
A2 = np.maximum(0, Z2)   # ReLU → (N, H2)

y_hat = A2 @ W3 + b3     # (N, 1)

loss = 0.5 * np.mean((y_hat - y) ** 2)

# ---------- 反向传播 ----------
dy_hat = (y_hat - y) / N   # (N, 1)

# 输出层梯度
dW3 = A2.T @ dy_hat        # (H2, 1)
db3 = np.sum(dy_hat, axis=0, keepdims=True)  # (1, 1)

# 第二隐藏层梯度
dA2 = dy_hat @ W3.T        # (N, H2)
dZ2 = dA2 * (Z2 > 0)       # ReLU'(Z2)
dW2 = A1.T @ dZ2           # (H1, H2)
db2 = np.sum(dZ2, axis=0, keepdims=True)  # (1, H2)

# 第一隐藏层梯度
dA1 = dZ2 @ W2.T           # (N, H1)
dZ1 = dA1 * (Z1 > 0)       # ReLU'(Z1)
dW1 = X.T @ dZ1            # (D, H1)
db1 = np.sum(dZ1, axis=0, keepdims=True)  # (1, H1)

# ---------- 参数更新 ----------
W3 -= lr * dW3
b3 -= lr * db3
W2 -= lr * dW2
b2 -= lr * db2
W1 -= lr * dW1
b1 -= lr * db1

# ---------- 输出 ----------
print("Loss:", loss)
print("\nUpdated W1:\n", W1)
print("Updated b1:\n", b1)
print("\nUpdated W2:\n", W2)
print("Updated b2:\n", b2)
print("\nUpdated W3:\n", W3)
print("Updated b3:\n", b3)

### 为以上代码添加多轮训练循环机制
- 输入维度扩展到了20，隐藏层维度扩展为50
- 训练轮数通过超参数epochs进行控制

**提示**：可通过手动调整如下代码中的epoch值（例如0.01 -> 0.02 -> 0.05 -> 0.1），来验证不同学习率下模型训练的收敛速度。

In [ ]:
import numpy as np

# 设置随机种子，确保可重复性
np.random.seed(42)

# ---------- 模拟数据 ----------
N, D_in, H1, H2, D_out = 256, 20, 50, 50, 1  # 样本数量256，输入维度20，隐藏层同为50，输出维度为1

# 输入 X 和真实标签 y，形状分别为 (N, D_in) 和 (N, 1)
X = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# ---------- 权重初始化 ----------
W1 = np.random.randn(D_in, H1) * 0.01
b1 = np.zeros((1, H1))

W2 = np.random.randn(H1, H2) * 0.01
b2 = np.zeros((1, H2))

W3 = np.random.randn(H2, D_out) * 0.01
b3 = np.zeros((1, D_out))

# ---------- 超参数 ----------
learning_rate = 0.05
epochs = 500

# ---------- 激活函数 ----------
def relu(x):
    return np.maximum(0, x)

def relu_grad(x):
    return (x > 0).astype(float)

# ---------- 训练循环 ----------
for epoch in range(epochs):
    # ---- 前向传播 ----
    z1 = X @ W1 + b1      # (N, H1)
    a1 = relu(z1)         # (N, H1)

    z2 = a1 @ W2 + b2     # (N, H2)
    a2 = relu(z2)         # (N, H2)

    z3 = a2 @ W3 + b3     # (N, D_out)
    y_pred = z3           # 输出层无激活函数（线性输出）

    # ---- 计算损失 ----
    loss = np.mean((y_pred - y) ** 2)

    if epoch % 50 == 0:
        print(f"Epoch {epoch:3d} | Loss: {loss:.6f}")

    # ---- 反向传播 ----
    dloss = 2 * (y_pred - y) / N   # MSE 损失的导数

    # 第三层梯度
    dW3 = a2.T @ dloss             # (H2, D_out)
    db3 = np.sum(dloss, axis=0, keepdims=True)  # (1, D_out)

    # 第二层反传
    da2 = dloss @ W3.T            # (N, H2)
    dz2 = da2 * relu_grad(z2)     # (N, H2)

    dW2 = a1.T @ dz2              # (H1, H2)
    db2 = np.sum(dz2, axis=0, keepdims=True)

    # 第一层反传
    da1 = dz2 @ W2.T              # (N, H1)
    dz1 = da1 * relu_grad(z1)     # (N, H1)

    dW1 = X.T @ dz1               # (D_in, H1)
    db1 = np.sum(dz1, axis=0, keepdims=True)

    # ---- 参数更新 ----
    W3 -= learning_rate * dW3
    b3 -= learning_rate * db3

    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1

# ---------- 最终损失 ----------
print(f"Final Loss: {loss:.6f}")

### Mini-Batch机制：将样本按批次进行训练

**原因**

将样本分批次进行训练（mini-batch 训练）的主要原因包括：
- 计算效率：降低内存需求，适合大型数据集，充分利用硬件并行性。
- 梯度稳定性：提供比单一样本更稳定的梯度估计，同时比全批量更高效。
- 泛化能力：通过随机打乱和批次处理，增强模型的鲁棒性和泛化能力。
- 灵活性：可调整 batch 大小，适应不同规模数据集和硬件条件。
- 更快收敛：更频繁的参数更新有助于快速探索参数空间。

下面的示例代码中，mini-batch训练通过将$ N = 256 $个样本分成8个batch（每个32个样本），实现了高效且稳定的训练过程。

In [ ]:
import numpy as np

# 设置随机种子，确保可重复性
np.random.seed(42)

# ---------- 模拟数据 ----------
N, D_in, H1, H2, D_out = 256, 20, 50, 50, 1  # 样本数量256，输入维度20，隐藏层同为50，输出维度为1
batch_size = 32  # 每个mini-batch的大小

# 输入 X 和真实标签 y，形状分别为 (N, D_in) 和 (N, 1)
X = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# ---------- 权重初始化 ----------
W1 = np.random.randn(D_in, H1) * 0.01
b1 = np.zeros((1, H1))
W2 = np.random.randn(H1, H2) * 0.01
b2 = np.zeros((1, H2))
W3 = np.random.randn(H2, D_out) * 0.01
b3 = np.zeros((1, D_out))

# ---------- 超参数 ----------
learning_rate = 0.1
epochs = 500
num_batches = N // batch_size  # 计算 batch 数量

# ---------- 激活函数 ----------
def relu(x):
    return np.maximum(0, x)

def relu_grad(x):
    return (x > 0).astype(float)

# ---------- 训练循环 ----------
for epoch in range(epochs):
    # 打乱数据（每个epoch随机化样本顺序）
    indices = np.random.permutation(N)
    X_shuffled = X[indices]
    y_shuffled = y[indices]
    
    # 初始化 epoch 总损失
    epoch_loss = 0.01
    
    # 按mini-batch迭代
    for i in range(num_batches):
        # 获取当前 batch 的数据
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        X_batch = X_shuffled[start_idx:end_idx]  # (batch_size, D_in)
        y_batch = y_shuffled[start_idx:end_idx]  # (batch_size, D_out)
        
        # ---- 前向传播 ----
        z1 = X_batch @ W1 + b1      # (batch_size, H1)
        a1 = relu(z1)               # (batch_size, H1)
        z2 = a1 @ W2 + b2           # (batch_size, H2)
        a2 = relu(z2)               # (batch_size, H2)
        z3 = a2 @ W3 + b3           # (batch_size, D_out)
        y_pred = z3                 # 输出层无激活函数（线性输出）
        
        # ---- 计算损失 ----
        batch_loss = np.mean((y_pred - y_batch) ** 2)
        epoch_loss += batch_loss
        
        # ---- 反向传播 ----
        dloss = 2 * (y_pred - y_batch) / batch_size  # MSE 损失的导数
        
        # 第三层梯度
        dW3 = a2.T @ dloss          # (H2, D_out)
        db3 = np.sum(dloss, axis=0, keepdims=True)  # (1, D_out)
        
        # 第二层反传
        da2 = dloss @ W3.T          # (batch_size, H2)
        dz2 = da2 * relu_grad(z2)   # (batch_size, H2)
        
        dW2 = a1.T @ dz2            # (H1, H2)
        db2 = np.sum(dz2, axis=0, keepdims=True)
        
        # 第一层反传
        da1 = dz2 @ W2.T            # (batch_size, H1)
        dz1 = da1 * relu_grad(z1)   # (batch_size, H1)
        
        dW1 = X_batch.T @ dz1       # (D_in, H1)
        db1 = np.sum(dz1, axis=0, keepdims=True)
        
        # ---- 参数更新 ----
        W3 -= learning_rate * dW3
        b3 -= learning_rate * db3
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2
        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1
    
    # 计算平均 epoch 损失
    epoch_loss /= num_batches
    
    # 每 50 个 epoch 输出损失
    if epoch % 50 == 0:
        print(f"Epoch {epoch:3d} | Loss: {epoch_loss:.6f}")

# ---------- 最终损失（全数据集） ----------
z1 = X @ W1 + b1
a1 = relu(z1)
z2 = a1 @ W2 + b2
a2 = relu(z2)
z3 = a2 @ W3 + b3
y_pred = z3
final_loss = np.mean((y_pred - y) ** 2)
print(f"Final Loss: {final_loss:.6f}")

### 正则化机制：降低过拟合风险
正则化是神经网络训练中防止模型过拟合的核心机制，其核心思想是通过对模型施加约束，限制其复杂度，从而提升在未知数据上的泛化能力。

**正则化的主要作用**：

1. 抑制过拟合：减少模型对训练数据噪声的敏感度
2. 提升泛化性：增强模型对新数据的适应能力
3. 特征选择与稀疏化（L1）：将不重要特征的权重压缩至0，实现自动特征筛选
4. 平滑权重分布（L2）：使所有权重趋近于较小值，避免极端权重
5. 加速训练：简化模型结构，优化收敛效率

**常用的正则化方法**：
| 正则化方法             | 描述                                                                 | 公式表示                              | 特点                                                                 |
|------------------------|----------------------------------------------------------------------|---------------------------------------|----------------------------------------------------------------------|
| **L1正则化（Lasso）**  | 在损失函数中加入权重的L1范数惩罚项，鼓励模型产生稀疏权重              | $J(\theta) = \text{Loss} + \lambda \sum_{i} \vert\theta_i\vert$ | 实现特征选择，产生稀疏模型，但可能导致解的不稳定性                |
| **L2正则化（Ridge）**  | 在损失函数中加入权重的L2范数惩罚项，防止权重过大                      | $J(\theta) = \text{Loss} + \lambda \sum_{i} \theta_i^2$         | 防止权重过大，提升模型稳定性，适用于多重共线性问题      |
| **Dropout**            | 训练中随机丢弃部分神经元，减少神经元间的共适应性                     | -                                     | 强迫网络学习冗余特征，类似集成学习效果，但增加训练时间   |
| **早停法（Early Stopping）** | 根据验证集性能停止训练，防止过度拟合训练数据                     | -                                     | 简单易行，需验证集监控，可能错过最佳模型                |
| **数据增强（Data Augmentation）** | 对训练数据随机变换（如旋转/裁剪），扩展数据集多样性          | -                                     | 提升模型鲁棒性，尤其适用于图像/文本数据，但增加计算开销 |
| **批归一化（Batch Normalization）** | 对每层输入归一化处理，稳定数据分布                        | -                                     | 加速训练并间接正则化，减少对初始权重的敏感度       |
| **权重衰减（Weight Decay）** | 优化器中加入权重衰减项，等价于L2正则化                          | 同L2正则化                            | 直接约束权重幅度，与优化器（如AdamW）结合使用              |
| **剪枝（Pruning）**    | 移除训练后不重要的连接或神经元，压缩模型规模                         | -                                     | 减少推理计算量，适合资源受限设备，但需额外训练步骤              |

下面示例代码中使用了L2正则化机制

**L2正则化机制**

1. **L2正则化的定义**

- L2正则化在损失函数中添加权重平方和的惩罚项，新的损失函数为：$L = \text{MSE} + \lambda \left( \|W_1\|_2^2 + \|W_2\|_2^2 + \|W_3\|_2^2 \right)$

  其中：
  - $\text{MSE} = \frac{1}{\text{batch\_size}} \sum_{i=1}^{\text{batch\_size}} (y_{\text{pred},i} - y_i)^2$ 是均方误差。
  - $\|W_k\|_2^2 = \sum_{i,j} W_{k,ij}^2$ 是权重矩阵的 L2 范数的平方。
  - $\lambda = \text{l2\_lambda}$ 是正则化系数，控制正则化强度。
  - 
- 代码中添加了超参数l2_lambda = 0.01，表示正则化强度。

2. **损失计算中的改动**
   损失代码中添加了L2正则化项。
   ```python
   l2_loss = l2_lambda * (np.sum(W1 ** 2) + np.sum(W2 ** 2) + np.sum(W3 ** 2))
   batch_loss = mse_loss + l2_loss
   ```
3. **梯度计算中的改动**
   - L2 正则化对权重 $ W_k $ 的梯度贡献为：$$\frac{\partial}{\partial W_k} \left( \lambda \|W_k\|_2^2 \right) = 2 \lambda W_k$$
   - 因此，权重梯度需要加上 L2 正则化项：
     - 原梯度：dW3 = a2.T @ dloss
     - 新梯度：dW3 = a2.T @ dloss + 2 * l2_lambda * W3
     - 类似地，dW2 和 dW1 也增加正则化项
   - 偏置（b1, b2, b3）不参与 L2 正则化，梯度计算不变。

4. **正则化的效果**
   - L2 正则化通过惩罚大权重值，鼓励模型学习较小的权重，减少模型复杂度，降低过拟合风险。
   - 代码中设置 l2_lambda = 0.01，这是一个适中的正则化强度，可以根据实验效果调整（例如 0.001 或 0.1）。


 


In [ ]:
import numpy as np

# 设置随机种子，确保可重复性
np.random.seed(42)

# ---------- 模拟数据 ----------
N, D_in, H1, H2, D_out = 256, 20, 50, 50, 1  # 样本数量256，输入维度20，隐藏层同为50，输出维度为1
batch_size = 32  # 每个 mini-batch 的大小

# 输入 X 和真实标签 y，形状分别为 (N, D_in) 和 (N, 1)
X = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# ---------- 权重初始化 ----------
W1 = np.random.randn(D_in, H1) * 0.01
b1 = np.zeros((1, H1))
W2 = np.random.randn(H1, H2) * 0.01
b2 = np.zeros((1, H2))
W3 = np.random.randn(H2, D_out) * 0.01
b3 = np.zeros((1, D_out))

# ---------- 超参数 ----------
learning_rate = 0.01
epochs = 500
num_batches = N // batch_size  # 计算 batch 数量
l2_lambda = 0.01  # L2 正则化系数

# ---------- 激活函数 ----------
def relu(x):
    return np.maximum(0, x)

def relu_grad(x):
    return (x > 0).astype(float)

# ---------- 训练循环 ----------
for epoch in range(epochs):
    # 打乱数据（每个 epoch 随机化样本顺序）
    indices = np.random.permutation(N)
    X_shuffled = X[indices]
    y_shuffled = y[indices]
    
    # 初始化 epoch 总损失
    epoch_loss = 0.0
    
    # 按 mini-batch 迭代
    for i in range(num_batches):
        # 获取当前 batch 的数据
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        X_batch = X_shuffled[start_idx:end_idx]  # (batch_size, D_in)
        y_batch = y_shuffled[start_idx:end_idx]  # (batch_size, D_out)
        
        # ---- 前向传播 ----
        z1 = X_batch @ W1 + b1      # (batch_size, H1)
        a1 = relu(z1)               # (batch_size, H1)
        z2 = a1 @ W2 + b2           # (batch_size, H2)
        a2 = relu(z2)               # (batch_size, H2)
        z3 = a2 @ W3 + b3           # (batch_size, D_out)
        y_pred = z3                 # 输出层无激活函数（线性输出）
        
        # ---- 计算损失（包括 L2 正则化项） ----
        mse_loss = np.mean((y_pred - y_batch) ** 2)
        # L2 正则化项：lambda * (||W1||^2 + ||W2||^2 + ||W3||^2)
        l2_loss = l2_lambda * (np.sum(W1 ** 2) + np.sum(W2 ** 2) + np.sum(W3 ** 2))
        batch_loss = mse_loss + l2_loss
        epoch_loss += batch_loss
        
        # ---- 反向传播 ----
        # MSE 损失的导数
        dloss = 2 * (y_pred - y_batch) / batch_size
        
        # 第三层梯度（加上 L2 正则化项）
        dW3 = a2.T @ dloss + 2 * l2_lambda * W3  # (H2, D_out)
        db3 = np.sum(dloss, axis=0, keepdims=True)  # (1, D_out)
        
        # 第二层反传
        da2 = dloss @ W3.T          # (batch_size, H2)
        dz2 = da2 * relu_grad(z2)   # (batch_size, H2)
        
        dW2 = a1.T @ dz2 + 2 * l2_lambda * W2  # (H1, H2)
        db2 = np.sum(dz2, axis=0, keepdims=True)
        
        # 第一层反传
        da1 = dz2 @ W2.T            # (batch_size, H1)
        dz1 = da1 * relu_grad(z1)   # (batch_size, H1)
        
        dW1 = X_batch.T @ dz1 + 2 * l2_lambda * W1  # (D_in, H1)
        db1 = np.sum(dz1, axis=0, keepdims=True)
        
        # ---- 参数更新 ----
        W3 -= learning_rate * dW3
        b3 -= learning_rate * db3
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2
        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1
    
    # 计算平均 epoch 损失
    epoch_loss /= num_batches
    
    # 每 50 个 epoch 输出损失
    if epoch % 50 == 0:
        print(f"Epoch {epoch:3d} | Loss: {epoch_loss:.6f}")

# ---------- 最终损失（全数据集） ----------
z1 = X @ W1 + b1
a1 = relu(z1)
z2 = a1 @ W2 + b2
a2 = relu(z2)
z3 = a2 @ W3 + b3
y_pred = z3
mse_loss = np.mean((y_pred - y) ** 2)
l2_loss = l2_lambda * (np.sum(W1 ** 2) + np.sum(W2 ** 2) + np.sum(W3 ** 2))
final_loss = mse_loss + l2_loss
print(f"Final Loss: {final_loss:.6f}")

## 基于PyTorch的神经网络构建与训练

### PyTorch的前向传播、反向传播与计算图
- 计算函数y=ReLU(Wx+b)，并对其进行反向传播和求梯度

PyTorch直接提供了神经网络训练中用到的前向传播、反向传播、激活函数、损失函数和参数优化（优化器）等组件中的多种主流实现，我们可以在代码中直接调用而无须再手动进行编码实现。

| **组件**               | **功能**                                                                 | **核心类/函数示例**                                 | **底层实现依赖**               |
| :--------------------- | :----------------------------------------------------------------------- | :-------------------------------------------------- | :----------------------------- |
| **Tensor**             | 多维数组，支持 GPU 加速和自动微分                                           | `torch.tensor([[1, 2], [3, 4]], dtype=torch.float)` | ATen 张量库（C++）     |
| **Autograd**           | 自动微分引擎，动态构建计算图并计算梯度                                       | `x.requires_grad=True; y.backward()`                | Autograd 引擎（C++）    |
| **nn.Module**          | 神经网络模块基类，封装层、损失函数等                                         | `class CNN(nn.Module): def __init__(self): ...`     | pybind11 绑定（Python-C++ 接口） |
| **Optimizer**          | 优化算法库（SGD、Adam 等），更新模型参数                                     | `torch.optim.Adam(model.parameters(), lr=0.001)`    | C++ 优化器后端             |
| **DataLoader**         | 高效数据加载，支持批处理、乱序、并行加载                                      | `DataLoader(dataset, batch_size=64, shuffle=True)` | 多进程管理（C++）      |
| **Distributed**        | 分布式训练支持（多 GPU/多节点）                                             | `DistributedDataParallel(model)`                    | gloo/NCCL 通信库（C++） |
| **TorchScript**        | 模型编译为静态图，提升部署性能                                                | `scripted_model = torch.jit.script(model)`          | JIT 编译器（C++）      |
| **扩展库**             | 领域专用工具库（CV/NLP/音频）                                               | `torchvision.models.resnet50()`                     | 独立 C++ 模块          |

In [ ]:
import torch
import torch.nn.functional as F

# 定义输入、权重、偏置和目标值
x = torch.tensor([[1.0, 2.0, 3.0]], requires_grad=True)  # 输入张量维度 (1, 3)
W = torch.tensor([[0.5, -0.2, 0.1], [0.3, 0.4, -0.5]], requires_grad=True)  # 权重矩阵维度 (2, 3)
b = torch.tensor([[0.1], [0.2]], requires_grad=True)  # 偏置向量 (2, 1)
target = torch.tensor([[1.0], [0.5]])  # 目标值 (2, 1)

# 前向传播
# 示例函数：y=ReLU(Wx + b)
def forward(x, W, b):
    # 线性变换: z = Wx + b
    z = torch.matmul(W, x.T) + b  # 矩阵乘法 + 偏置
    # ReLU激活: y = ReLU(z)
    y = torch.relu(z)
    return y

# 执行前向传播
y = forward(x, W, b)
print("Forward output (y):", y)

# 计算MSE损失
loss = F.mse_loss(y, target)
print("MSE Loss:", loss.item())

# 反向传播
loss.backward()

# 输出梯度
print("Gradient of x:", x.grad)
print("Gradient of W:", W.grad)
print("Gradient of b:", b.grad)

### 计算图的相关图形

依赖于操作系统上安装好graphviz包，例如对于Ubuntu系统，可以使用如下命令进行。

```bash
sudo apt-get update
sudo apt-get install graphviz
```

In [ ]:
from torchviz import make_dot
y = forward(x, W, b)
loss = F.mse_loss(y, target)
make_dot(loss).render("computational_graph", format="png")

### 基于PyTorch的神经网络构建与训练
- 将前面示例“多层神经网络”修改为基于PyTorch的实现。


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 设置随机种子，确保可重复性
torch.manual_seed(42)

# ---------- 模拟数据 ----------
# 将 NumPy 数组转换为 PyTorch 张量
X = torch.tensor([[1.0, 2.0], [3.0, 4.0]], dtype=torch.float32)  # shape: (2, 2)
y = torch.tensor([[5.0], [11.0]], dtype=torch.float32)          # shape: (2, 1)

N, D = X.shape

# 网络结构参数
H1 = 4  # 第 1 个隐藏层神经元个数
H2 = 4  # 第 2 个隐藏层神经元个数
lr = 0.01 # 学习率

# ---------- 定义神经网络模型 ----------
# PyTorch 的 nn.Module 是构建神经网络的基础
class SimpleNN(nn.Module):
    def __init__(self, input_size, H1_size, H2_size, output_size):
        super(SimpleNN, self).__init__()
        # 定义各层
        # nn.Linear(in_features, out_features) 自动包含权重和偏置
        self.fc1 = nn.Linear(input_size, H1_size)
        self.relu1 = nn.ReLU() # ReLU 激活函数
        self.fc2 = nn.Linear(H1_size, H2_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(H2_size, output_size)

    def forward(self, x):
        # 定义前向传播路径
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

# 实例化模型
model = SimpleNN(D, H1, H2, 1)

# ---------- 定义损失函数和优化器 ----------
# 均方误差损失 (MSELoss)
criterion = nn.MSELoss(reduction='mean') # reduction='mean'表示取批次的平均损失
# Adam优化器，它会自动处理参数的梯度更新
optimizer = optim.Adam(model.parameters(), lr=lr)

# ---------- 单次训练迭代 ----------
print("--- 进行单次训练迭代 ---")

# 前向传播
y_hat = model(X)

# 计算损失
# PyTorch的MSELoss已经处理了0.5和求平均
loss = criterion(y_hat, y)

# 反向传播和优化
optimizer.zero_grad() # 清除之前的梯度，避免累积
loss.backward()       # 执行反向传播，计算所有可训练参数的梯度
optimizer.step()      # 根据计算出的梯度更新模型参数

print(f"Loss: {loss.item():.4f}")

# ---------- 输出更新后的参数 (可选) ----------
# 注意：PyTorch 权重的形状通常是 (out_features, in_features)，与 NumPy 中的 (in_features, out_features) 相反
print("\n--- 更新后的参数 ---")
print("W1 (fc1.weight):\n", model.fc1.weight.data)
print("b1 (fc1.bias):\n", model.fc1.bias.data)
print("\nW2 (fc2.weight):\n", model.fc2.weight.data)
print("b2 (fc2.bias):\n", model.fc2.bias.data)
print("\nW3 (fc3.weight):\n", model.fc3.weight.data)
print("b3 (fc3.bias):\n", model.fc3.bias.data)


# ---------- 验证最终预测 ----------
with torch.no_grad(): # 在评估模式下，不计算梯度
    y_pred_final = model(X)

print("\n--- 最终预测 ---")
print("原始数据 X:\n", X)
print("原始标签 y:\n", y)
print("预测值 y_hat:\n", y_pred_final)

### 基于神经网络的多轮训练示例

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# 设置随机种子，确保可重复性
torch.manual_seed(42)
np.random.seed(42)

# ---------- 模拟数据 ----------
N, D_in, H1, H2, D_out = 256, 20, 50, 50, 1  # 样本数量256，输入维度20，隐藏层同为50，输出维度为1
batch_size = 32  # 每个 mini-batch 的大小

# 输入X和真实标签y，转换为PyTorch张量
X = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 创建PyTorch数据集和 DataLoader
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)  # 随机打乱

# ---------- 定义神经网络模型 ----------
class NeuralNetwork(nn.Module):
    def __init__(self, D_in, H1, H2, D_out):
        super(NeuralNetwork, self).__init__()
        # 第一隐藏层
        self.linear1 = nn.Linear(D_in, H1)
        self.relu1 = nn.ReLU()
        # 第二隐藏层
        self.linear2 = nn.Linear(H1, H2)
        self.relu2 = nn.ReLU()
        # 输出层
        self.linear3 = nn.Linear(H2, D_out)
        
        # 使用He初始化权重
        nn.init.kaiming_normal_(self.linear1.weight, mode='fan_in', nonlinearity='relu')
        nn.init.zeros_(self.linear1.bias)
        nn.init.kaiming_normal_(self.linear2.weight, mode='fan_in', nonlinearity='relu')
        nn.init.zeros_(self.linear2.bias)
        nn.init.kaiming_normal_(self.linear3.weight, mode='fan_in', nonlinearity='relu')
        nn.init.zeros_(self.linear3.bias)
    
    def forward(self, x):
        # 前向传播
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        return x

# ---------- 超参数 ----------
learning_rate = 0.01
epochs = 500
l2_lambda = 0.01  # L2正则化系数

# ---------- 初始化模型、损失函数和优化器 ----------
model = NeuralNetwork(D_in, H1, H2, D_out)
criterion = nn.MSELoss(reduction='mean')  # 均方误差损失
# 使用SGD优化器，weight_decay实现L2正则化
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=l2_lambda)

# ---------- 训练循环 ----------
model.train()  # 设置模型为训练模式
for epoch in range(epochs):
    epoch_loss = 0.0
    num_batches = len(dataloader)
    
    # 遍历 mini-batches
    for X_batch, y_batch in dataloader:
        # 前向传播
        y_pred = model(X_batch)
        
        # 计算损失（MSE损失，L2正则化由优化器自动处理）
        mse_loss = criterion(y_pred, y_batch)
        epoch_loss += mse_loss.item()
        
        # 反向传播和优化
        optimizer.zero_grad()  # 清空梯度
        mse_loss.backward()    # 计算梯度
        optimizer.step()       # 更新参数
    
    # 计算平均 epoch 损失
    epoch_loss /= num_batches
    
    # 每 50 个 epoch 输出损失
    if epoch % 50 == 0:
        print(f"Epoch {epoch:3d} | Loss: {epoch_loss:.6f}")

# ---------- 最终损失（全数据集） ----------
model.eval()  # 设置模型为推理模式
with torch.no_grad():
    y_pred = model(X)
    mse_loss = criterion(y_pred, y)
    # 计算 L2 正则化项
    l2_loss = 0.0
    for param in model.parameters():
        if param.requires_grad and len(param.shape) > 1:  # 仅对权重矩阵应用L2正则化
            l2_loss += torch.sum(param ** 2)
    l2_loss *= l2_lambda
    final_loss = mse_loss + l2_loss
    print(f"Final Loss: {final_loss.item():.6f}")

## 张量及其自动求导

在PyTorch的世界里，张量（Tensor） 是所有操作的基石，就像NumPy中的ndarray一样。然而，PyTorch的张量远不止一个多维数组那么简单，它还承载着在GPU上加速计算以及自动求导的核心能力，这些都是深度学习的命脉。

简单来说，张量就是一个多维数组，可以存储各种类型的数据，从简单的标量（一个数字）到复杂的图像、视频数据，甚至神经网络的权重和梯度。

### 张量创建

张量支持多种创建方式，包括从已有数据创建、使用函数创建、从文件加载等。

#### 从 Python 列表或 NumPy 数组创建

In [ ]:
import torch
import numpy as np

# 从Python列表创建
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)
print("从列表创建:\n", x_data)

# 从NumPy数组创建
np_array = np.array(data)
x_np = torch.from_numpy(np_array) # 注意：会共享内存
print("从NumPy数组创建:\n", x_np)

#### 创建全0或全1张量

In [ ]:
# 2 行 3 列的全一张量
x_ones = torch.ones(2, 3)
print("全一张量:\n", x_ones)

# 2 行 3 列的全零张量
x_zeros = torch.zeros(2, 3)
print("全零张量:\n", x_zeros)

# 指定值
x_full = torch.full((2, 3), 7)
print("全指定值张量:\n", x_full)

# 单位矩阵/对角矩阵
x_eye = torch.eye(3)
x_diag = torch.diag(torch.tensor([1, 2, 3]))
print("单位矩阵:\n", x_eye)
print("对角矩阵:\n", x_diag)

#### 创建随机张量

In [ ]:
# 2 行 3 列的随机张量 (0到1之间均匀分布)
x_rand = torch.rand(2, 3)
print("均匀分布随机张量:\n", x_rand)

# 2行3列的随机张量 (标准正态分布)
x_randn = torch.randn(2, 3)
print("正态分布随机张量:\n", x_randn)

# [0,10)之间的随机数构成的张量，维度2x3
torch.randint(0, 10, (2, 3))

#### 特定分布初始化
torch.normal()可用于生成符合正态分布（高斯分布）的随机数张量，它支持多种参数组合，可根据需求灵活生成不同均值、标准差和形状的随机数据。

**核心参数**
- mean：正态分布的均值（标量或张量）。
- std：正态分布的标准差（标量或张量）。
- size：输出张量的形状（元组），当 mean 和 std 均为标量时必须指定。
- generator：控制随机数生成的种子（可选）。
- out：指定输出张量（可选）

In [ ]:
# 生成形状为 (2, 3) 的标准正态分布随机数（均值为0，标准差为1）
tensor = torch.normal(mean=0.0, std=1.0, size=(2, 3))
print("准正态分布随机数（均值为0，标准差为1）的随机张量:\n", tensor)

# 使用generator参数（随机数种子）确保结果可复现
gen = torch.Generator().manual_seed(42)
tensor = torch.normal(mean=0, std=1, size=(2,2), generator=gen)
print("准正态分布随机数（均值为0，标准差为1）的随机张量，可复现:\n", tensor)

# 使用示例1：神经网络权重初始化
weights = torch.normal(mean=0.0, std=0.01, size=(100, 200))  # 高斯初始化

# 使用示例2：数据增强（添加噪声）
clean_data = torch.randn(10, 3, 224, 224)  # 原始数据
noisy_data = clean_data + torch.normal(mean=0, std=0.1, size=clean_data.shape)   # .shape方法用于获取指定tensor的形状

# 生成线性回归数据集：y = Xw + b + 噪声
X = torch.normal(0, 1, (100, 2))
y = X @ torch.tensor([3.0, -2.0]) + 1.5 + torch.normal(0, 0.1, (100,))
print("生成的回归数据集:\n", y)

#### 使用 arange()/linspace()/logspace()
- torch.arange()：创建等差数列（整数或浮点数）
- torch.linspace()：创建均匀间隔的线性序列
- torch.logspace()：创建对数尺度上的均匀序列

**常用参数**
- start：指数起始值
- end：指数结束值（包含）
- steps：生成的元素个数
- base：对数底数（默认10），logspace()专用
- dtype：数据类型

In [ ]:
# 浮点数序列：1.0到4.0（步长0.5）
a = torch.arange(1, 4, 0.5)             # tensor([1.0, 1.5, 2.0, 2.5, 3.0, 3.5])
# 指定设备和数据类型
b = torch.arange(0, 6, 2, dtype=torch.float32, device='cpu')  # tensor([0., 2., 4.], device='cuda:0')

# 生成0到1之间的5个等分点
c = torch.linspace(0, 1, 5)             # tensor([0.0000, 0.2500, 0.5000, 0.7500, 1.0000])
# 生成-1到1之间的3个点
d = torch.linspace(-1, 1, 3)             # tensor([-1., 0., 1.])

# 生成10^0到10^2之间的5个点（底数10）
e = torch.logspace(0, 2, 5)             # tensor([1.0000, 3.1623, 10.0000, 31.6228, 100.0000])

#### 克隆和复制
创建的新张量与原张量数据相同，但不共享内存，修改副本不影响原张量。

In [ ]:
a = torch.tensor([1, 2, 3])
b = a.clone()
b[0] = 100
print(a)  # tensor([1, 2, 3])  原张量不变
print(b)  # tensor([100, 2, 3])

#### 从文件加载Tensor

首先生成示例CSV文件。

In [ ]:
import numpy as np

# 创建2D数值数组（例如3x3矩阵）
data = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

# 保存为CSV，控制格式和分隔符
np.savetxt("example_csv_data.csv", data, 
           delimiter=",",   # 逗号分隔
           fmt="%d",        # 整数格式（避免科学计数）
           #header="Col1,Col2,Col3",  # 自定义列名
           comments="")     # 避免header被注释

#### 从csv加载tensor示例

需要首先由NumPy加载，而后再使用.from_numpy()方法转换为Tensor。

In [ ]:
# 读取CSV（无标题行）
array = np.loadtxt("example_csv_data.csv", delimiter=",")
# 所有字段必须为数值型，否则.float()方法可能会失败
tensor = torch.from_numpy(array).float()

print(tensor.shape)  # e.g., [3, 3]

#### 加载图像文件为Tensor

适用于计算视觉类的任务，需要通过torchvision模型的read_image进行。

首先，我们要先安装好torchvision模块。

In [ ]:
!pip install torchvision torchaudio

In [ ]:
from torchvision.io import read_image

img_tensor = read_image("computational_graph.png")  # 结果为[C, H, W]格式，uint8类型
img_tensor = img_tensor.float() / 255.0  # 转为float并归一化

print(img_tensor.shape)  # e.g., [3, 224, 224]

#### 加载音频文件为Tensor

In [ ]:
import torchaudio

waveform, sample_rate = torchaudio.load("audio.wav")  # waveform是 [channels, time]

print(waveform.shape)      # e.g., [1, 16000]
print(sample_rate)         # 采样率

### 张量支持的操作

#### 索引与切片

In [ ]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]])

print(x[0])        # 第一行
print(x[:, 1])     # 第二列
print(x[1, 2])     # 第2行第3列的元素
print(x[:, 1:])    # 所有行，第2列到最后

# 高级索引
indices = torch.tensor([0, 2])
print(x[0, indices])  # 第1行中的第1列和第3列

#### 变形操作

In [ ]:
x = torch.arange(12)

print("原始:", x)
print("reshape:", x.view(3, 4))
print("添加维度:", x.unsqueeze(0).shape)
print("去除维度:", x.unsqueeze(0).squeeze().shape)
print("转置:", torch.randn(2, 3).t())

#### 连接操作

In [ ]:
a = torch.ones(2, 3)
b = torch.zeros(2, 3)

print("拼接 (dim=0):", torch.cat([a, b], dim=0))  # 4x3
print("拼接 (dim=1):", torch.cat([a, b], dim=1))  # 2x6

# 堆叠（增加新维度）
print("堆叠:", torch.stack([a, b], dim=0))  # 2x2x3

#### 数学操作

In [ ]:
x = torch.tensor([1.0, 2.0, 3.0])
y = torch.tensor([4.0, 5.0, 6.0])

print("加:", x + y)
print("乘:", x * y)
print("点积:", torch.dot(x, y))
print("矩阵乘:", torch.matmul(x.unsqueeze(0), y.unsqueeze(1)))  # 1x1
print("指数:", torch.exp(x))
print("对数:", torch.log(x))

#### 聚合操作

In [ ]:
x = torch.tensor([[1, 2], [3, 4]])

print("总和:", x.sum())
print("均值:", x.float().mean())
print("最小/最大:", x.min(), x.max())
print("按行求和:", x.sum(dim=1))

### 自动求导机制

#### 自动求导机制示例

In [ ]:
## 第一部分：NumPy与PyTorch Tensor

import torch
import numpy as np

# 从 NumPy 创建 Tensor
np_array = np.array([[1.0, 2.0], [3.0, 4.0]])
tensor_from_np = torch.from_numpy(np_array)
print("NumPy转Tensor：\n", tensor_from_np)

# 从 Tensor 创建 NumPy
tensor = torch.tensor([[5.0, 6.0], [7.0, 8.0]])
np_from_tensor = tensor.numpy()
print("Tensor转NumPy：\n", np_from_tensor)

# 修改会同步（共享内存）
tensor_from_np[0, 0] = 99
print("修改后的NumPy：\n", np_array)

## 第二部分：什么是自动求导机制？

# NumPy不支持自动求导，只能手动写导数
def f_np(x):
    return x**2 + 3*x + 5

x_val = 2.0
delta = 1e-5
numerical_grad = (f_np(x_val + delta) - f_np(x_val - delta)) / (2 * delta)
print("数值求导结果（NumPy手动）：", numerical_grad)

## 第三部分：PyTorch自动求导机制

# PyTorch 中的自动求导机制
x = torch.tensor([2.0], requires_grad=True)
y = x**2 + 3 * x + 5
y.backward()  # 自动反向传播
print("y对x的梯度值：", x.grad)

# 自动求导支持更复杂的模型
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
w = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
b = torch.tensor([0.5], requires_grad=True)

# 线性模型
y_pred = x @ w + b
loss = y_pred.sum()
loss.backward()

print("w的梯度：", w.grad)
print("b的梯度：", b.grad)

#### Tensor上的grad_fn属性

- 每个具有requires_grad=True的张量（除了用户自己创建的叶子张量外）都有一个**grad_fn**属性，该属性是非叶子张量专有
- 该属性指向一个函数，这个函数记录了创建当前张量所用的操作，并且知道如何计算该操作的梯度



以下脚本执行后返回的MulBackward0和AddBackward0就是PyTorch内部用来计算梯度链式法则的函数。

In [ ]:
import torch

x = torch.tensor([1.0], requires_grad=True) # 叶子张量
print(f"x.grad_fn: {x.grad_fn}")

y = x * 2 # 非叶子张量，由乘法操作生成
print(f"y.grad_fn: {y.grad_fn}")

z = y + 3 # 非叶子张量，由加法操作生成
print(f"z.grad_fn: {z.grad_fn}")

#### grad属性

loss.backward()被调用后，requires_grad=True的**叶子张量**的.grad属性会累积其梯度值。

In [ ]:
import torch

x = torch.tensor([2.0], requires_grad=True)
y = x**2 + 5 * x + 3
y.backward() # 计算梯度

print(f"x 的梯度: {x.grad}") # dy/dx = 2x + 5 = 2*2 + 5 = 9

#### 自动求导的过程示例

为了详细说明PyTorch的自动求导机制，以函数$ y = W(\text{ReLU}(WX + b)) + b $为例，我们将分析哪些是叶张量（leaf tensors）、哪些是非叶张量（non-leaf tensors），以及自动求导的完整过程。

In [ ]:
import torch

# 定义张量
X = torch.randn(2, 3, requires_grad=True)    # 输入张量，形状 (2, 3)
W1 = torch.randn(4, 3, requires_grad=True)   # 第一层权重，形状 (4, 3)
b1 = torch.randn(4, requires_grad=True)      # 第一层偏置，形状 (4,)
W2 = torch.randn(4, 4, requires_grad=True)   # 第二层权重，形状 (4, 4)
b2 = torch.randn(4, requires_grad=True)      # 第二层偏置，形状 (4,)

# 前向计算
z = torch.matmul(X, W1.t()) + b1             # W1X + b1，形状 (2, 4)

z_relu = torch.relu(z)                       # ReLU(W1X + b1)，形状 (2, 4)

y = torch.matmul(z_relu, W2) + b2            # W2(ReLU(W1X + b1)) + b2，形状 (2, 4)

loss = y.sum()                               # 标量损失

# 反向传播
loss.backward()

# 打印叶张量信息
print("X.is_leaf:", X.is_leaf)               # True
print("W1.is_leaf:", W1.is_leaf)             # True
print("b1.is_leaf:", b1.is_leaf)             # True
print("W2.is_leaf:", W2.is_leaf)             # True
print("b2.is_leaf:", b2.is_leaf)             # True
print("z.is_leaf:", z.is_leaf)               # False
print("z_relu.is_leaf:", z_relu.is_leaf)     # False
print("y.is_leaf:", y.is_leaf)               # False
print("loss.is_leaf:", loss.is_leaf)         # False

# 打印非叶张量的grad_fn
print(f"非叶张量z的grad_fn:\n{z.grad_fn}")
print(f"非叶张量z_relu的grad_fn:\n{z_relu.grad_fn}")
print(f"非叶张量y的grad_fn:\n{y.grad_fn}")

# 打印叶张量的grad
print("X.grad:", X.grad)                     # 梯度
print("W1.grad:", W1.grad)
print("b1.grad:", b1.grad)
print("W2.grad:", W2.grad)
print("b2.grad:", b2.grad)

## 构建并训练神经网络

### 构建和训练神经网络的基本步骤

**构建神经网络的基本步骤**
- 准备数据：使用torch.utils.data加载和预处理数据
- 定义网络结构：继承 nn.Module 类，定义网络的层和前向传播逻辑
- 初始化网络：设置层的参数（如输入/输出维度）和激活函数
- 选择损失函数和优化器：如交叉熵损失和SGD 优化器
- 训练网络：通过前向传播、计算损失、反向传播和参数更新完成训练
- 评估模型：在测试集上验证模型性能


#### 第一步：定义神经网络结构
- 目标：设计神经网络的架构，指定层、激活函数和其他组件。
- 主要模块：
  - torch.nn.Module：所有神经网络的基类，用于定义自定义网络。
  - torch.nn：提供构建块，如全连接层 (nn.Linear)、卷积层 (nn.Conv2d)、激活函数 (nn.ReLU)、池化层 (nn.MaxPool2d) 等。
  - torch.nn.Sequential：用于快速构建按顺序连接的层。
- 步骤：
  - 继承 nn.Module，在 __init__ 中定义层，在 forward 方法中实现前向传播逻辑。
  - 将层注册为类属性以自动跟踪参数。

In [ ]:
import torch
import torch.nn as nn

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # 输入 784，输出 128
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)   # 输出 10 个类别

    def forward(self, x):
        x = x.view(-1, 784)  # 展平输入
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#### 第二步：准备数据
- 目标：加载和预处理数据集，组织为适合训练的格式。
- 主要模块：
  - torch.utils.data.Dataset：定义数据集的接口，用于加载数据。
  - torch.utils.data.DataLoader：批量加载数据，支持打乱和多线程。
  - torchvision.datasets：提供常用数据集（如 MNIST、CIFAR-10）。
  - torchvision.transforms：提供数据预处理和增强工具。
- 步骤：
  - 使用 Dataset 或现成数据集加载数据。
  - 应用变换（如归一化、数据增强）。
  - 使用DataLoader创建批量迭代器。

In [ ]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.ToTensor()
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

#### 第三步：初始化模型、损失函数和优化器
- 目标：设置模型实例、损失函数和优化算法。
- 主要模块：
  - torch.nn：提供损失函数，如 nn.CrossEntropyLoss（分类）、nn.MSELoss（回归）。
  - torch.optim：提供优化器，如 optim.SGD、optim.Adam。
- 步骤：
  - 实例化网络并移动到设备（CPU/GPU）。
  - 选择适合任务的损失函数。
  - 配置优化器，传入模型参数和超参数（如学习率）。

In [ ]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#### 第四步：训练模型
- 目标：通过前向传播、损失计算、反向传播和参数更新训练模型。
- 主要模块：
  - torch.nn.Module：通过 model(input) 执行前向传播。
  - torch.autograd：自动计算梯度（隐式使用）。
  - torch.optim：优化器更新参数。
- 步骤：
  - 遍历数据集，执行以下循环：
    1. 清零梯度（optimizer.zero_grad()）。
    2. 前向传播，计算输出。
    3. 计算损失（criterion(output, target)）。
    4. 反向传播（loss.backward()）。
    5. 更新参数（optimizer.step()）。

In [ ]:
model.train()
for epoch in range(5):
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}')

#### 第五步：评估模型
- 目标：在测试集上验证模型性能。
- 主要模块：
  - torch.nn.Module：使用 model.eval() 切换到评估模式。
  - torch.no_grad：禁用梯度计算，节省内存。
- 步骤：
  - 加载测试数据。
  - 设置模型为评估模式（model.eval()）。
  - 使用 torch.no_grad() 计算预测并评估指标（如准确率）。

In [ ]:
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Test Accuracy: {100 * correct / total:.2f}%')

#### 第六步：保存模型
- 目标：保存训练好的模型参数或整个模型，并在需要时加载。
- 主要模块：
  - torch.nn.Module：提供 state_dict() 和 load_state_dict() 方法。
  - torch.save 和 torch.load：用于序列化和反序列化。
- 步骤：
  - 保存模型参数（推荐）或整个模型。
  - 加载参数并恢复模型。

In [ ]:
# 保存模型
torch.save(model.state_dict(), 'model.pth')

#### 加载模型的简单示例

In [ ]:
# 加载模型
model = SimpleNet().to(device)
model.load_state_dict(torch.load('model.pth'))
model.eval()

### 完整的示例
不包含模型的保存和加载。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 定义神经网络
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # 输入层到隐藏层（784 -> 128）
        self.relu = nn.ReLU()               # 激活函数
        self.fc2 = nn.Linear(128, 64)       # 隐藏层（128 -> 64）
        self.fc3 = nn.Linear(64, 10)        # 输出层（64 -> 10，10个类别）

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # 展平输入（batch_size, 1, 28, 28）-> (batch_size, 784)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)          # 输出 logits
        return x

# 准备数据
transform = transforms.ToTensor()
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# 初始化网络、损失函数和优化器
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 训练网络
for epoch in range(5):  # 训练 5 个 epoch
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()          # 清零梯度
        outputs = model(images)        # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        loss.backward()               # 反向传播
        optimizer.step()              # 更新参数
        
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}')

print('训练完成！')

### PyTorch模型组合示例

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#### 使用nn.Sequential构建顺序网络

In [ ]:
class SequentialModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
    
    def forward(self, x):
        return self.model(x)

# 测试
x = torch.randn(4, 784)
seq_model = SequentialModel()
print("Sequential output shape:", seq_model(x).shape)

#### 使用 nn.ModuleList 构建重复结构网络

In [ ]:
class ModuleListModel(nn.Module):
    def __init__(self, num_layers=3):
        super().__init__()
        self.input_layer = nn.Linear(784, 256)
        self.hidden_layers = nn.ModuleList([
            nn.Linear(256, 256) for _ in range(num_layers)
        ])
        self.output_layer = nn.Linear(256, 10)
    
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
        return self.output_layer(x)

# 测试
x = torch.randn(4, 784)
list_model = ModuleListModel()
print("ModuleList output shape:", list_model(x).shape)

#### 使用 nn.ModuleDict 构建分支结构网络

In [ ]:
class ModuleDictModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.shared = nn.Linear(784, 256)
        self.heads = nn.ModuleDict({
            'classification': nn.Linear(256, 10),
            'regression': nn.Linear(256, 1)
        })
    
    def forward(self, x, mode='classification'):
        x = F.relu(self.shared(x))
        return self.heads[mode](x)

# 测试
x = torch.randn(4, 784)
dict_model = ModuleDictModel()
print("ModuleDict classification head:", dict_model(x, mode='classification').shape)
print("ModuleDict regression head:", dict_model(x, mode='regression').shape)

### 激活函数示例

**主流激活函数的特性对比**

| 特性/函数        | Softmax                                    | Sigmoid                                  | Tanh                                     | ReLU                                   | GELU                                           | Swish                                          |
| :--------------- | :----------------------------------------- | :--------------------------------------- | :--------------------------------------- | :------------------------------------- | :--------------------------------------------- | :--------------------------------------------- |
| **数学表达式** | $P_i = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$ | $\sigma(x) = \frac{1}{1 + e^{-x}}$       | $\tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$ | $f(x) = \max(0, x)$                    | $f(x) = x \cdot P(X \le x)$ (标准正态CDF)       | $f(x) = x \cdot \sigma(\beta x)$               |
| **输出范围** | $(0, 1)$ (所有输出和为 1)                  | $(0, 1)$                                 | $(-1, 1)$                                | $[0, +\infty)$                         | $(-\infty, +\infty)$                           | $(-\infty, +\infty)$                           |
| **零中心输出** | 否（单独看每个输出）                       | 否                                       | **是** | 否                                     | **是** (输出均值趋近于 0)                  | **是** (输出均值趋近于 0)                  |
| **平滑性** | 平滑                                       | 平滑                                     | 平滑                                     | 不平滑 (在 $x=0$ 处尖锐)             | **平滑** | **平滑** |
| **单调性** | 无法直接判断 (是整个向量的转换)              | **单调递增** | **单调递增** | **单调递增** | **非单调** | **非单调** |
| **是否存在梯度消失** | 否（通常不饱和），但有数值稳定性问题       | **是** (两端饱和区)                      | **是** (两端饱和区)                      | **否** (正值区)，但有“死亡 ReLU” | **否** (正值区，平滑过渡)，但计算复杂 | **否** (正负值区都有梯度)，但计算复杂 |
| **“死亡神经元”问题** | 不适用                                     | 不适用                                   | 不适用                                   | **是** (负值区梯度为 0)                | 否                                             | 否                                             |
| **计算成本** | 高 (涉及指数和求和)                        | 高 (涉及指数)                            | 高 (涉及指数)                            | **最低** (比较操作)                    | 较高 (涉及 $\tanh$ 和多项式)                  | 较高 (涉及 Sigmoid 和乘法)                     |
| **典型应用场景** | **多分类问题**的**输出层**，输出类别概率   | **二分类问题**的**输出层**，多标签分类 | 早期神经网络的**隐藏层**，RNNs             | **大多数隐藏层**，CNNs                 | **Transformer 架构** (BERT, GPT), 复杂模型  | **深层网络**，视觉任务 (图像分类、目标检测)    |
| **主要优点** | 将输出转换为概率分布，适用于互斥多分类       | 输出概率化，可微                           | 输出零中心，收敛快于 Sigmoid           | 缓解梯度消失，计算高效，引入稀疏性     | 性能优越，平滑，自适应门控，缓解梯度消失       | 性能优越，平滑，非单调，自门控，无死亡 ReLU |
| **主要缺点** | 数值稳定性，可能导致梯度接近 0               | 梯度消失，计算成本高，非零中心输出       | 梯度消失，计算成本高                     | “死亡 ReLU”，非零中心输出              | 计算成本高，概念复杂                           | 计算成本高                                     |

#### 示例1：使用不带激活函数的模型拟合纯性数据点

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np

# ========== Step 1: 构造数据 ==========
np.random.seed(0)
x_np = np.linspace(-3, 3, 200)
noise = np.random.normal(0, 0.3, x_np.shape)
y_np = 2 * x_np + 1 + noise

X_tensor = torch.tensor(x_np, dtype=torch.float32).view(-1, 1)
Y_tensor = torch.tensor(y_np, dtype=torch.float32).view(-1, 1)

# ========== Step 2: 定义模型 ==========
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)  # y = wx + b

    def forward(self, x):
        return self.linear(x)

model = LinearRegressionModel()

# ========== Step 3: 定义损失函数和优化器 ==========
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.05)

# ========== Step 4: 训练并记录每隔几步的预测 ==========
epochs = 200
history = []

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_tensor)
    loss = criterion(output, Y_tensor)
    loss.backward()
    optimizer.step()

    # 每 5 轮记录一次当前预测值
    if epoch % 5 == 0 or epoch == epochs - 1:
        with torch.no_grad():
            pred_y = model(X_tensor).detach()
            history.append(pred_y.numpy())

print("训练完成，准备动画...")

# ========== Step 5: 动态可视化拟合过程 ==========
fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(x_np, y_np, c='blue', label="Data")
line, = ax.plot([], [], 'r-', linewidth=2, label="Prediction")
text = ax.text(0.05, 0.95, '', transform=ax.transAxes, fontsize=12, verticalalignment='top')
ax.set_xlim(-3, 3)
ax.set_ylim(min(y_np)-1, max(y_np)+1)
ax.legend()
ax.set_title("Linear Regression Fitting Animation")

def init():
    line.set_data([], [])
    text.set_text('')
    return line, text

def animate(i):
    y_pred = history[i].flatten()
    line.set_data(x_np, y_pred)
    loss_val = np.mean((y_np - y_pred)**2)
    text.set_text(f"Epoch: {i*5:>3}, Loss: {loss_val:.4f}")
    return line, text

ani = animation.FuncAnimation(fig, animate, init_func=init,
                              frames=len(history), interval=150, blit=True)

plt.close()  # 防止重复显示
ani

# 显示动画（在Jupyter中）
HTML(ani.to_jshtml())

#### 示例2：使用不带激活函数的模型拟合非线性数据点

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np

# ========== Step 1: 生成非线性数据 ==========
np.random.seed(0)
x_np = np.linspace(-3, 3, 200)
noise = np.random.normal(0, 0.5, x_np.shape)
#y_np = x_np**2 + noise  # 非线性关系示例1：幂函数
y_np = np.sin(x_np * 2) + noise  # 非线性关系示例2：三角函数

X_tensor = torch.tensor(x_np, dtype=torch.float32).view(-1, 1)
Y_tensor = torch.tensor(y_np, dtype=torch.float32).view(-1, 1)

# ========== Step 2: 定义线性模型 ==========
class LinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)  # 线性模型 y = wx + b

    def forward(self, x):
        return self.linear(x)

model = LinearModel()

# ========== Step 3: 损失函数与优化器 ==========
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.05)

# ========== Step 4: 训练并记录过程 ==========
epochs = 200
history = []

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_tensor)
    loss = criterion(output, Y_tensor)
    loss.backward()
    optimizer.step()

    # 每 5 epoch 记录一次
    if epoch % 5 == 0 or epoch == epochs - 1:
        with torch.no_grad():
            pred_y = model(X_tensor).detach()
            history.append(pred_y.numpy())

print("训练完成，准备动画...")

# ========== Step 5: Matplotlib 动画可视化 ==========
fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(x_np, y_np, c='blue', label="Nonlinear Data")
line, = ax.plot([], [], 'r-', linewidth=2, label="Linear Fit")
text = ax.text(0.05, 0.95, '', transform=ax.transAxes, fontsize=12, verticalalignment='top')
ax.set_xlim(-3, 3)
ax.set_ylim(min(y_np) - 1, max(y_np) + 1)
ax.legend()
ax.set_title("Linear Model Fitting Nonlinear Data")

def init():
    line.set_data([], [])
    text.set_text('')
    return line, text

def animate(i):
    y_pred = history[i].flatten()
    line.set_data(x_np, y_pred)
    loss_val = np.mean((y_np - y_pred)**2)
    text.set_text(f"Epoch: {i*5:>3}, Loss: {loss_val:.4f}")
    return line, text

ani = animation.FuncAnimation(fig, animate, init_func=init,
                              frames=len(history), interval=150, blit=True)

plt.close()

# 显示动画（在Jupyter中）
HTML(ani.to_jshtml())

#### 示例3：在模型上添加ReLU激活函数并拟合非线性数据点

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np

# ========== Step 1: 生成非线性数据 ==========
np.random.seed(0)
x_np = np.linspace(-3, 3, 200)
noise = np.random.normal(0, 0.5, x_np.shape)
#y_np = x_np**2 + noise  # 非线性关系示例1：幂函数
y_np = np.sin(x_np * 2) + noise  # 非线性关系示例2：三角函数

X_tensor = torch.tensor(x_np, dtype=torch.float32).view(-1, 1)
Y_tensor = torch.tensor(y_np, dtype=torch.float32).view(-1, 1)

# ========== Step 2: 定义非线性模型（带激活函数） ==========
class NonlinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.net(x)

model = NonlinearModel()

# ========== Step 3: 损失函数与优化器 ==========
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.05)

# ========== Step 4: 训练并记录过程 ==========
epochs = 200
history = []

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_tensor)
    loss = criterion(output, Y_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 5 == 0 or epoch == epochs - 1:
        with torch.no_grad():
            pred_y = model(X_tensor).detach()
            history.append(pred_y.numpy())

print("训练完成，准备动画...")

# ========== Step 5: 动态可视化拟合过程 ==========
fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(x_np, y_np, c='skyblue', label="Nonlinear Data")
line, = ax.plot([], [], 'r-', linewidth=2, label="Model Fit")
text = ax.text(0.05, 0.95, '', transform=ax.transAxes, fontsize=12, verticalalignment='top')
ax.set_xlim(-3, 3)
ax.set_ylim(min(y_np) - 1, max(y_np) + 1)
ax.legend()
ax.set_title("MLP with ReLU Fitting Nonlinear Data")

def init():
    line.set_data([], [])
    text.set_text('')
    return line, text

def animate(i):
    y_pred = history[i].flatten()
    line.set_data(x_np, y_pred)
    loss_val = np.mean((y_np - y_pred)**2)
    text.set_text(f"Epoch: {i*5:>3}, Loss: {loss_val:.4f}")
    return line, text

ani = animation.FuncAnimation(fig, animate, init_func=init,
                              frames=len(history), interval=150, blit=True)

plt.close()

# 显示动画（在Jupyter中）
HTML(ani.to_jshtml())

### torch.nn的层

In [ ]:
import torch.nn as nn
dir(nn)  # 输出 nn 模块下所有类名

### 损失函数示例

#### 损失函数基础应用示例

In [ ]:
import torch
import torch.nn as nn

# 1. 定义损失函数
criterion = nn.MSELoss(reduction='mean')

# 2. 准备数据，手动提供input（预测值）
input = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], requires_grad=True)
target = torch.tensor([[1.5, 2.5, 3.5], [3.5, 4.5, 5.5]])

# 3. 计算损失
loss = criterion(input, target)

# 4. 反向传播
loss.backward()

# 5. 输出损失
print(f"Loss (mean): {loss.item()}")

# 可选：逐元素损失
criterion_none = nn.MSELoss(reduction='none')  # 不进行聚合，输出则为与input/target同形状的张量
loss_none = criterion_none(input, target)
print(f"Element-wise loss:\n{loss_none}")

# 验证求和损失（reduction='sum'）
criterion_sum = nn.L1Loss(reduction='sum')
loss_sum = criterion_sum(input, target)
print(f"Sum loss: {loss_sum.item()}")

In [ ]:
import torch
import torch.nn as nn

# 定义损失函数
criterion = nn.L1Loss(reduction='mean')  # 默认均值

# 示例数据
input = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], requires_grad=True)
target = torch.tensor([[1.5, 2.5, 3.5], [3.5, 4.5, 5.5]])

# 计算损失
loss = criterion(input, target)

# 输出损失
print(f"Loss (mean): {loss.item()}")  # 平均绝对误差

# 反向传播
loss.backward()

# 验证逐元素损失（reduction='none'）
criterion_none = nn.L1Loss(reduction='none')
loss_none = criterion_none(input, target)
print(f"Element-wise loss:\n{loss_none}")

# 验证求和损失（reduction='sum'）
criterion_sum = nn.L1Loss(reduction='sum')
loss_sum = criterion_sum(input, target)
print(f"Sum loss: {loss_sum.item()}")

#### Huber损失示例

In [ ]:
import torch
import torch.nn as nn

# 定义损失函数
criterion = nn.SmoothL1Loss(reduction='mean', beta=1.0)

# 示例数据
input = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], requires_grad=True)
target = torch.tensor([[1.5, 2.5, 3.5], [3.5, 4.5, 5.5]])

# 计算损失
loss = criterion(input, target)

# 输出损失
print(f"Loss (mean): {loss.item()}")  # 平滑 L1 损失

# 反向传播
loss.backward()

# 验证逐元素损失（reduction='none'）
criterion_none = nn.SmoothL1Loss(reduction='none', beta=1.0)
loss_none = criterion_none(input, target)
print(f"Element-wise loss:\n{loss_none}")

# 验证求和损失（reduction='sum'）
criterion_sum = nn.SmoothL1Loss(reduction='sum', beta=1.0)
loss_sum = criterion_sum(input, target)
print(f"Sum loss: {loss_sum.item()}")

### 分类损失函数

| **损失函数**                | **输入要求**                                                                 | **目标要求**                          | **主要功能**                              | **适用场景**                                                                 | **优点**                                                                 | **局限**                                                                 | **PyTorch 实现**                     |
|-----------------------------|-----------------------------------------------------------------------------|---------------------------------------|------------------------------------------|-----------------------------------------------------------------------------|--------------------------------------------------------------------------|--------------------------------------------------------------------------|--------------------------------------|
| **CrossEntropyLoss**        | Logits（未经过 softmax），形状 \([N, C, ...]\)                               | 类别索引（整数），形状 \([N, ...]\)    | 结合 log-softmax 和 NLL，计算多分类交叉熵 | 单标签多分类（如图像分类、文本分类）                                         | 数值稳定，适合标准多分类任务                                              | 不支持多标签或软标签                                                     | `torch.nn.CrossEntropyLoss`          |
| **BCEWithLogitsLoss**       | Logits（未经过 Sigmoid），形状任意                                           | 二值标签（0/1 或概率），形状与输入相同 | 结合 Sigmoid 和二元交叉熵，计算二分类损失 | 单标签/多标签二分类、图像分割、GAN 判别器                                    | 数值稳定，支持多标签和不平衡数据（通过 `pos_weight`）                      | 仅限二分类，不适合多分类                                                 | `torch.nn.BCEWithLogitsLoss`         |
| **BCELoss**                 | 概率（经过 Sigmoid，[0, 1]），形状任意                                       | 二值标签（0/1 或概率），形状与输入相同 | 计算二元交叉熵损失                       | 二分类、多标签分类（需手动 Sigmoid）                                         | 适合已计算概率的场景                                                     | 数值稳定性稍差，需手动 Sigmoid                                           | `torch.nn.BCELoss`                   |
| **NLLLoss**                 | Log 概率（经过 log-softmax），形状 \([N, C, ...]\)                           | 类别索引（整数），形状 \([N, ...]\)    | 计算负对数似然损失                       | 多分类（需手动 log-softmax）、语言模型                                      | 适合自定义 log-softmax 的场景                                             | 需要手动 log-softmax，不如 CrossEntropyLoss 方便                          | `torch.nn.NLLLoss`                   |
| **KLDivLoss**               | Log 概率（通常 log-softmax 输出），形状任意                                  | 目标概率分布，形状与输入相同           | 计算 KL 散度（交叉熵的一部分）            | 分布匹配、软标签分类、知识蒸馏、VAE                                          | 适合软标签和分布优化                                                     | 需要目标概率分布，计算复杂                                               | `torch.nn.KLDivLoss`                 |

#### BCELoss示例

In [ ]:
import torch
import torch.nn as nn

# 模拟预测值（需要事先经过sigmoid激活）
input = torch.tensor([0.9, 0.2, 0.8], requires_grad=True)

# 真实标签（必须是0或1）
target = torch.tensor([1.0, 0.0, 1.0])

# 初始化损失函数
criterion = nn.BCELoss()

# 计算损失
loss = criterion(input, target)

print("BCELoss:", loss.item())

# 反向传播
loss.backward()

#### BCEWithLogitsLoss示例

In [ ]:
import torch
import torch.nn as nn

# logits（未经过sigmoid）
input = torch.tensor([0.2, -1.0, 3.0], requires_grad=True)

# 标签值
target = torch.tensor([1.0, 0.0, 1.0])

# 定义损失函数
criterion = nn.BCEWithLogitsLoss()

# 计算损失
loss = criterion(input, target)
print("BCEWithLogitsLoss:", loss.item())

# 反向传播
loss.backward()

#### NLLLoss示例

In [ ]:
import torch
import torch.nn as nn

# 实例化 NLLLoss，使用默认的 reduction='mean'
nll_loss = nn.NLLLoss()

# 示例 1: 简单多分类，batch_size=2
# 模拟模型输出的原始 logits
raw_logits = torch.tensor([
    [0.1, 2.0, 0.3],  # 样本1
    [1.5, 0.2, 0.8]   # 样本2
], dtype=torch.float32)

# 必须先经过LogSoftmax激活函数
log_probabilities = torch.log_softmax(raw_logits, dim=1)

# 真实类别标签 (整数索引)
targets = torch.tensor([1, 0], dtype=torch.long) # 样本1真实类别是1，样本2真实类别是0

loss = nll_loss(log_probabilities, targets)
print(f"NLLLoss (mean): {loss.item()}") # 输出: NLLLoss (mean): 0.2422...

# 示例 2: 使用 weight 参数处理类别不平衡
# 假设类别0权重1.0，类别1权重10.0（少数类），类别2权重1.0
class_weights = torch.tensor([1.0, 10.0, 1.0], dtype=torch.float32)
nll_loss_weighted = nn.NLLLoss(weight=class_weights)

# 损失计算将根据真实类别对应的权重进行加权平均
loss_weighted = nll_loss_weighted(log_probabilities, targets)
print(f"NLLLoss (with class weights): {loss_weighted.item()}") # 输出: NLLLoss (with class weights): 1.0675...

#### CrossEntropyLoss示例

In [ ]:
import torch
import torch.nn as nn

# 实例化 CrossEntropyLoss，使用默认的 reduction='mean'
cross_entropy_loss = nn.CrossEntropyLoss()

# 示例 1: 简单多分类，batch_size=2，3个类别
# input 是模型输出的原始 logits (不需要Softmax)
# 形状: (batch_size, num_classes)
predictions_logits = torch.tensor([
    [0.1, 2.0, 0.3],  # 样本1
    [1.5, 0.2, 0.8]   # 样本2
], dtype=torch.float32)

# 真实类别标签，形状: (batch_size,)，是整数索引
targets = torch.tensor([1, 0], dtype=torch.long) # 样本1真实类别是1，样本2真实类别是0

loss = cross_entropy_loss(predictions_logits, targets)
print(f"CrossEntropyLoss (mean): {loss.item()}") # 输出: CrossEntropyLoss (mean): 0.2422...

# 示例 2: 使用 weight 参数处理类别不平衡
# 假设类别0权重1.0，类别1权重10.0（少数类），类别2权重1.0
class_weights = torch.tensor([1.0, 10.0, 1.0], dtype=torch.float32)
cross_entropy_loss_weighted = nn.CrossEntropyLoss(weight=class_weights)

loss_weighted = cross_entropy_loss_weighted(predictions_logits, targets)
print(f"CrossEntropyLoss (with class weights): {loss_weighted.item()}") # 输出: CrossEntropyLoss (with class weights)

# 示例 3: 带有ignore_index的多维输入 (例如在语义分割中)
# 假设图像分割的logits形状为 (batch_size, num_classes, H, W)
# target 形状为 (batch_size, H, W)，包含像素级别的类别索引
predictions_segmentation_logits = torch.randn(2, 3, 4, 4) # 批次2，3个类别，4x4图像
targets_segmentation = torch.randint(0, 3, (2, 4, 4), dtype=torch.long)
# 假设类别 0 是要忽略的背景
targets_segmentation[0, 0, 0] = 0 # 将某个位置的标签设为要忽略的索引
targets_segmentation[1, 2, 3] = 0 # 另一个位置也设为忽略

loss_fn_ignore = nn.CrossEntropyLoss(ignore_index=0) # 忽略类别0的损失
loss_segmentation = loss_fn_ignore(predictions_segmentation_logits, targets_segmentation)
print(f"CrossEntropyLoss (with ignore_index): {loss_segmentation.item()}")

### 优化器

| 优化策略                     | 简介                                                                 | 特点                                                                                     | 优点                                                                 | 缺点                                                                 |
|------------------------------|----------------------------------------------------------------------|------------------------------------------------------------------------------------------|----------------------------------------------------------------------|----------------------------------------------------------------------|
| **标准梯度下降 (GD)**        | 使用整个数据集计算梯度，一次更新所有参数。                           | 全量数据，梯度准确，计算成本高。                                                         | 稳定，收敛可靠（凸问题）。                                           | 计算慢，内存需求大，不适合大规模数据。                               |
| **随机梯度下降 (SGD)**       | 每次随机抽取一个样本计算梯度并更新参数。                             | 单样本更新，计算快，噪声大。                                                             | 效率高，适合大规模数据，噪声可帮助逃离局部极值。                     | 梯度波动大，收敛不稳定。                                             |
| **小批量梯度下降 (Mini-batch GD)** | 使用小批量样本计算梯度并更新参数。                                   | 平衡稳定性和效率，batch 大小通常为 32、64 等。                                           | 兼顾效率和稳定性，广泛应用。                                         | Batch 大小需调参，可能陷入局部极值。                                 |
| **动量法 (Momentum)**        | 引入动量项，累积历史梯度方向，加速收敛。                             | 平滑更新方向，动量参数通常为 0.9。                                                       | 加速收敛，减少震荡，适合深层网络。                                   | 需调参，可能超调。                                                   |
| **Nesterov 加速梯度 (NAG)**  | 动量法改进，提前计算前瞻梯度。                                       | 基于“未来”位置梯度更新，更精确。                                                         | 收敛更快，适合曲率大区域。                                           | 实现复杂，需调参。                                                   |
| **Adagrad**                  | 自适应调整学习率，根据历史梯度平方和缩放。                           | 稀疏特征更新快，频繁特征更新慢。                                                         | 适合稀疏数据，无需手动调学习率。                                     | 学习率过早衰减，可能停止更新。                                       |
| **RMSprop**                  | Adagrad 改进，用指数移动平均平滑梯度平方累积。                       | 防止学习率过早衰减，衰减率通常为 0.9。                                                   | 适合非平稳目标，收敛稳定。                                           | 需调整初始学习率。                                                   |
| **Adam**                     | 结合动量法和 RMSprop，维护一阶和二阶时刻的移动平均。                 | 自适应学习率，偏差校正，参数 β₁≈0.9，β₂≈0.999。                                          | 收敛快，适应性强，默认参数表现好。                                   | 泛化性能可能逊于 SGD，需调参。                                       |
| **AdamW**                    | Adam 改进，添加权重衰减以提高泛化能力。                             | 解耦权重衰减与自适应学习率。                                                             | 泛化性能优于 Adam，适合复杂任务。                                     | 需调权重衰减系数。                                                   |
| **AdaDelta**                 | RMSprop 扩展，无需设置全局学习率，用更新量移动平均缩放更新。         | 基于更新量移动平均调整，免去学习率设置。                                                 | 无需调学习率，适合复杂任务。                                         | 实现复杂，使用较少。                                                 |

**PyTorch实现的13优化算法**

| 优化算法       | 原理                                                                 | 关键参数                                                                 | 适用场景                                                   | 优点                                                                 | 缺点                                                                 |
|----------------|----------------------------------------------------------------------|--------------------------------------------------------------------------|------------------------------------------------------------|----------------------------------------------------------------------|----------------------------------------------------------------------|
| **SGD**        | 随机梯度下降，支持动量和 Nesterov 动量，基于小批量梯度更新。         | `lr`, `momentum`, `dampening`, `weight_decay`, `nesterov`                | 大规模数据集，需精细控制的任务（如图像分类）。              | 简单，泛化性能好，适合大规模数据，动量加速收敛。             | 需手动调学习率，收敛可能较慢，易受噪声影响。                 |
| **Adam**       | 结合一阶和二阶动量的自适应学习率方法，收敛快。                       | `lr`, `betas`, `eps`, `weight_decay`, `amsgrad`                          | 深度学习通用任务（如 CNN、RNN、Transformer）。              | 收敛快，默认参数鲁棒，适应多种问题。                         | 泛化性能可能逊于 SGD，需调参。                               |
| **AdamW**      | Adam 的改进，解耦权重衰减和自适应学习率，增强泛化。                 | `lr`, `betas`, `eps`, `weight_decay`                                    | 需要更好泛化的任务（如 NLP、计算机视觉）。                 | 泛化性能优于 Adam，收敛稳定。                                | 需调权重衰减系数，计算稍复杂。                               |
| **RMSprop**    | 用指数移动平均平滑梯度平方，防止学习率过早衰减。                     | `lr`, `alpha`, `eps`, `weight_decay`, `momentum`, `centered`             | 非平稳目标（如 RNN、强化学习）。                           | 适合非平稳问题，收敛稳定。                                   | 需调初始学习率，泛化性能可能一般。                           |
| **Adagrad**    | 自适应调整学习率，基于历史梯度平方和，适合稀疏数据。                 | `lr`, `lr_decay`, `weight_decay`, `eps`                                 | 稀疏数据任务（如推荐系统、NLP 词嵌入）。                   | 无需手动调学习率，适合稀疏梯度。                             | 学习率过早衰减，可能停止更新。                               |
| **Adadelta**   | RMSprop 扩展，用更新量移动平均代替全局学习率。                       | `lr`, `rho`, `eps`, `weight_decay`                                      | 复杂任务，需免调学习率（如深度网络）。                     | 无需设置全局学习率，适应性强。                               | 实现复杂，使用较少，收敛可能较慢。                           |
| **Adamax**     | Adam 变体，用梯度无穷范数代替二阶动量，适合稀疏梯度。               | `lr`, `betas`, `eps`, `weight_decay`                                    | 稀疏梯度场景（如 NLP）。                                   | 适合稀疏数据，计算简单。                                     | 应用范围较窄，泛化性能一般。                                 |
| **NAdam**      | 结合 Adam 和 Nesterov 动量，更新更精确。                             | `lr`, `betas`, `eps`, `weight_decay`, `momentum_decay`                   | 需高精度更新的任务（如 Transformer）。                     | 收敛更快，更新方向更精确。                                   | 计算复杂度稍高，需调参。                                     |
| **RAdam**      | Adam 的改进，通过自适应整流减少初期方差。                            | `lr`, `betas`, `eps`, `weight_decay`                                    | 需稳定初始训练的任务（如深度网络）。                       | 初期更稳定，收敛性能好。                                     | 应用较新，泛化性能需验证。                                   |
| **ASGD**       | 对 SGD 参数更新进行平均，适合凸问题。                                | `lr`, `lambd`, `alpha`, `t0`, `weight_decay`                            | 凸优化问题（如线性回归）。                                 | 适合凸问题，收敛稳定。                                       | 深度学习中较少使用，效率较低。                               |
| **LBFGS**      | 二阶优化，使用有限内存近似 Hessian，适合小数据集。                   | `lr`, `max_iter`, `max_eval`, `history_size`                            | 小数据集，凸优化问题。                                     | 高精度，适合简单问题。                                       | 计算密集，需 closure 函数，不适合深度学习大模型。             |
| **SparseAdam** | Adam 的稀疏版本，优化稀疏张量梯度。                                  | `lr`, `betas`, `eps`                                                    | 稀疏梯度任务（如嵌入层优化）。                             | 适合稀疏数据，内存效率高。                                   | 仅限稀疏张量，应用受限。                                     |
| **Rprop**      | 基于梯度符号调整步长，忽略梯度大小。                                 | `lr`, `etas`, `step_sizes`                                              | 全批量优化，简单问题。                                     | 简单，适合全批量任务。                                       | 不适合深度学习，收敛慢。                                     |

### 使用优化器的标准步骤

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. 定义模型，这里直接使用仅包含一个线性层的模型
model = nn.Linear(10, 1)

# 打印模型的初始参数
#for name, param in model.named_parameters():
#    print(f"Name: {name} | Shape: {param.shape} | Value:\n{param.data}")

# 2. 定义损失函数
criterion = nn.MSELoss()

# 3. 实例化优化器（比如SGD）
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 模拟训练数据
x = torch.randn(32, 10)
y = torch.randn(32, 1)

# 4. （训练循环）为简化代码，这里仅提供一次迭代
optimizer.zero_grad()   # 清空旧梯度
output = model(x)       # 前向传播
loss = criterion(output, y)  # 计算损失
loss.backward()         # 反向传播计算梯度
optimizer.step()        # 更新模型参数

# 训练后，打印模型的参数
#for name, param in model.named_parameters():
#    print(f"Name: {name} | Shape: {param.shape} | Value:\n{param.data}")

#### 标准的模型训练流程
- 重点在于优化器的使用步骤
- 定义了训练循环

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 1. 定义模型，通过继承nn.Module进行，仅提供了一个纯性层
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(10, 1)
    
    def forward(self, x):
        return self.fc(x)

model = SimpleNet()

# 2. 定义损失函数
criterion = nn.MSELoss()

# 3. 定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. 准备数据
X = torch.randn(100, 10)
y = torch.randn(100, 1)
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 5. 训练循环
num_epochs = 10
for epoch in range(num_epochs):
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()        # 清零梯度
        outputs = model(batch_X)     # 前向传播
        loss = criterion(outputs, batch_y)  # 计算损失
        loss.backward()             # 反向传播
        optimizer.step()            # 更新参数
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# 6. （可选）保存模型
#torch.save(model.state_dict(), 'model.pth')

### optim.SGD优化算法示例
- 比较SGD、动量SGD和Nesterov动量的参数收敛效果

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

# 定义要优化的函数 f(x) = (x-5)^2
def func(x):
    return (x - 5) ** 2

# 初始化参数的起始值
start_x = torch.tensor([0.0], requires_grad=True)

# 设置优化参数
learning_rate = 0.1
num_iterations = 50 # 迭代次数

# --- 实验一：标准 SGD (momentum = 0) ---
print("--- 标准 SGD (momentum = 0) ---")
x_sgd = start_x.clone().detach().requires_grad_(True) # 克隆起始值，用于SGD
optimizer_sgd = optim.SGD([x_sgd], lr=learning_rate, momentum=0)

sgd_history = [x_sgd.item()]

for i in range(num_iterations):
    optimizer_sgd.zero_grad() # 清零之前的梯度
    loss = func(x_sgd)        # 计算损失
    loss.backward()           # 反向传播计算梯度
    optimizer_sgd.step()      # 更新参数

    sgd_history.append(x_sgd.item())

    # if (i + 1) % 10 == 0:
    #     print(f"Iteration {i+1}: x = {x_sgd.item():.4f}, Loss = {loss.item():.4f}")

print(f"最终参数x (标准SGD): {x_sgd.item():.4f}")


# --- 实验二：动量 SGD (momentum = 0.9) ---
print("\n--- 动量 SGD (momentum = 0.9) ---")
x_momentum = start_x.clone().detach().requires_grad_(True) # 克隆起始值，用于动量SGD
# 注意：这里设置了 momentum 参数
optimizer_momentum = optim.SGD([x_momentum], lr=learning_rate, momentum=0.9)

momentum_history = [x_momentum.item()]

for i in range(num_iterations):
    optimizer_momentum.zero_grad() # 清零之前的梯度
    loss = func(x_momentum)     # 计算损失
    loss.backward()          # 反向传播计算梯度
    optimizer_momentum.step()   # 更新参数

    momentum_history.append(x_momentum.item())

    # if (i + 1) % 10 == 0:
    #     print(f"Iteration {i+1}: x = {x_momentum.item():.4f}, Loss = {loss.item():.4f}")

print(f"最终参数x (动量SGD): {x_momentum.item():.4f}")

# --- 实验三：Nesterov动量SGD (momentum = 0.9, nesterov = True) ---
print("\n--- Nesterov动量SGD (momentum = 0.9, nesterov = True) ---")
x_nesterov = start_x.clone().detach().requires_grad_(True) # 克隆起始值，用于 Nesterov SGD
# 注意：这里设置了 momentum 和 nesterov 参数
optimizer_nesterov = optim.SGD([x_nesterov], lr=learning_rate, momentum=0.9, nesterov=True)

nesterov_history = [x_nesterov.item()]

for i in range(num_iterations):
    optimizer_nesterov.zero_grad() # 清零之前的梯度
    loss = func(x_nesterov)     # 计算损失
    loss.backward()          # 反向传播计算梯度
    optimizer_nesterov.step()   # 更新参数

    nesterov_history.append(x_nesterov.item())

    # if (i + 1) % 10 == 0:
    #     print(f"Iteration {i+1}: x = {x_nesterov.item():.4f}, Loss = {loss.item():.4f}")

print(f"最终参数x (Nesterov动量SGD): {x_nesterov.item():.4f}")


# --- 绘制参数随迭代次数的变化图 ---
plt.figure(figsize=(10, 6))
plt.plot(sgd_history, label='Standard SGD (momentum=0)', marker='o', linestyle='-')
plt.plot(momentum_history, label='Momentum SGD (momentum=0.9)', marker='x', linestyle='--')
plt.plot(nesterov_history, label='Nesterov Momentum SGD (momentum=0.9)', marker='*', linestyle='-.')
plt.axhline(y=5.0, color='r', linestyle='-', label='Optimal x = 5.0') # 绘制目标值
plt.xlabel("Iteration")
plt.ylabel("Parameter x value")
plt.title("Parameter x Value vs. Iteration for different SGD variants")
plt.legend()
plt.grid(True)
plt.show()

### 自适应学习率优化算法

| 特性 / 算法 | Adagrad (Adaptive Gradient) | RMSprop (Root Mean Square Propagation) | AdaDelta |
| :---------------- | :--------------------------------------------------------------------------------------------------------------------------------------------- | :--------------------------------------------------------------------------------------------------------------------------------------------------------- | :----------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **核心思想** | 根据**过去所有梯度平方的累加和**来自适应地调整每个参数的学习率。 | 根据**过去梯度平方的指数加权移动平均**（Exponentially Weighted Moving Average, EWMA）来自适应地调整每个参数的学习率。 | 结合了梯度平方的 EWMA 和参数更新的 EWMA，**在大多数情况下无需手动设置学习率**。 |
| **学习率调整** | 频繁更新的参数学习率下降更快，不频繁更新的参数学习率下降更慢。 | 解决了 Adagrad 学习率单调递减过快的问题，能更灵活地适应非稳态和非凸问题。 | 彻底消除了对全局学习率的依赖，能够动态调整学习步长。 |
| **优点** | - 适用于**稀疏数据**（如 NLP 中的词嵌入），能有效提高低频特征的学习效率。<br>- 无需手动调整每维学习率。 | - 缓解了 Adagrad 学习率过早衰减的问题。<br>- 在循环神经网络（RNN）等任务上表现良好。<br>- 比 Adagrad 适用范围更广。 | - **无需手动设置学习率**。<br>- 训练后期学习率不会过小，有助于持续学习。<br>- 对学习率超参数不敏感。 |
| **缺点** | - **学习率会持续单调递减**，可能导致训练后期学习率过小，模型过早停止学习。 | - 仍然需要手动设置一个**全局学习率**。<br>- 缺乏对动量的内置支持（通常需与 Momentum 结合）。 | - 相较于 Adam，在某些任务上收敛速度可能略慢。<br>- 实现相对复杂。 |
| **主要超参数** | 初始学习率 ($\eta$)，常数 $\epsilon$（小常数，防止除零） | 初始学习率 ($\eta$)，衰减率 ($\rho$)，常数 $\epsilon$ | 衰减率 ($\rho$)，常数 $\epsilon$ |
| **数学公式** | $\mathbf{g}_t = \nabla L(\theta_{t-1})$ <br>$\mathbf{G}_t = \mathbf{G}_{t-1} + \mathbf{g}_t \odot \mathbf{g}_t$ <br>$\theta_t = \theta_{t-1} - \frac{\eta}{\sqrt{\mathbf{G}_t + \epsilon}} \odot \mathbf{g}_t$ | $\mathbf{g}_t = \nabla L(\theta_{t-1})$ <br>$\mathbf{E}[\mathbf{g}^2]_t = \rho \mathbf{E}[\mathbf{g}^2]_{t-1} + (1-\rho) \mathbf{g}_t \odot \mathbf{g}_t$ <br>$\theta_t = \theta_{t-1} - \frac{\eta}{\sqrt{\mathbf{E}[\mathbf{g}^2]_t + \epsilon}} \odot \mathbf{g}_t$ | $\mathbf{g}_t = \nabla L(\theta_{t-1})$ <br>$\mathbf{E}[\mathbf{g}^2]_t = \rho \mathbf{E}[\mathbf{g}^2]_{t-1} + (1-\rho) \mathbf{g}_t \odot \mathbf{g}_t$ <br>$\Delta \theta_t = - \frac{\sqrt{\mathbf{E}[\Delta \theta^2]_{t-1} + \epsilon}}{\sqrt{\mathbf{E}[\mathbf{g}^2]_t + \epsilon}} \odot \mathbf{g}_t$ <br>$\mathbf{E}[\Delta \theta^2]_t = \rho \mathbf{E}[\Delta \theta^2]_{t-1} + (1-\rho) \Delta \theta_t \odot \Delta \theta_t$ <br>$\theta_t = \theta_{t-1} + \Delta \theta_t$ |
| **备注** | - $\theta$: 模型参数<br>- $L$: 损失函数<br>- $\nabla L(\theta)$: 梯度<br>- $\eta$: 初始学习率<br>- $\mathbf{g}_t$: 当前梯度<br>- $\mathbf{G}_t$: 历史梯度平方的累加和（对角矩阵）<br>- $\mathbf{E}[\mathbf{g}^2]_t$: 梯度平方的指数加权移动平均<br>- $\mathbf{E}[\Delta \theta^2]_t$: 参数更新平方的指数加权移动平均<br>- $\rho$: 衰减率 (RMSprop 和 Adadelta 中的超参数，通常为 0.9 或 0.99)<br>- $\epsilon$: 极小常数，通常为 $10^{-8}$，防止分母为零<br>- $\odot$: 元素级乘法（Hadamard积） | | |

#### 示例：对比三个自适应学习算法

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# 设置随机种子
torch.manual_seed(42)

# 生成数据
X = torch.linspace(-5, 5, 1000).reshape(-1, 1)
y = X**2 + 0.5 * X + torch.randn(X.size()) * 0.5  # y = x^2 + 0.5x + noise
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 定义模型
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(1, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    
    def forward(self, x):
        return self.layers(x)

# 训练函数
def train_model(optimizer_class, optimizer_params, num_epochs=50):
    model = SimpleNet()
    criterion = nn.MSELoss()
    optimizer = optimizer_class(model.parameters(), **optimizer_params)
    
    losses = []
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch_X, batch_y in dataloader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * batch_X.size(0)
        losses.append(epoch_loss / len(dataset))
    return losses

# 优化器参数
optimizers = [
    (optim.Adagrad, {'lr': 0.01, 'eps': 1e-10}, 'Adagrad'),
    (optim.RMSprop, {'lr': 0.01, 'alpha': 0.99, 'eps': 1e-8}, 'RMSprop'),
    (optim.Adadelta, {'lr': 1.0, 'rho': 0.9, 'eps': 1e-6}, 'Adadelta')
]

# 训练并记录损失
results = {}
for opt_class, params, name in optimizers:
    losses = train_model(opt_class, params)
    results[name] = losses

# 可视化
plt.figure(figsize=(10, 6))
for name, losses in results.items():
    plt.plot(losses, label=name)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Adagrad, RMSprop和Adadelta的损失曲线')
plt.legend()
plt.grid(True)
plt.show()

### 自适应动量优化算法

| **优化算法** | **融合机制** | **PyTorch类名** | **关键特性** | **适用场景** |
|--------------|--------------|------------------|--------------|--------------|
| **Adam** | 动量法 + 自适应学习率（一阶和二阶矩的指数移动平均） | `torch.optim.Adam` | - 结合一阶动量（梯度均值）和二阶动量（梯度平方均值）<br>- 偏差修正确保早期迭代稳定性<br>- 默认参数（lr=0.001, betas=(0.9, 0.999)）鲁棒 | - 几乎所有深度学习任务（如CNN、RNN、Transformer）<br>- 非凸优化问题<br>- 通用首选优化器 |
| **AdamW** | 动量法 + 自适应学习率 + 解耦权重衰减 | `torch.optim.AdamW` | - 改进Adam的权重衰减，解耦正则化与梯度更新<br>- 默认weight_decay=0.01，增强泛化<br>- 支持AMSGrad变体 | - 需要强正则化的任务（如Transformer、BERT）<br>- 大型模型或易过拟合场景 |
| **AMSGrad** | 动量法 + 自适应学习率 + 二阶动量最大值 | `Adam(amsgrad=True)` 或<br> `AdamW(amsgrad=True)` | - 维护二阶动量历史最大值，增强收敛稳定性<br>- 解决Adam在某些非凸问题中的不稳定<br>- 无需单独类，通过`amsgrad=True`启用 | - 非凸优化问题<br>- Adam表现不稳定或稀疏梯度场景 |
| **NAdam** | 动量法（Nesterov加速） + 自适应学习率 | `torch.optim.NAdam` | - 引入Nesterov动量，提前预测梯度方向<br>- 加速收敛，理论上优于Adam<br>- 增加momentum_decay参数（默认0.004） | - NLP任务（如RNN、Transformer）<br>- 需要快速收敛的场景 |
| **RAdam** | 动量法 + 自适应学习率 + 动态二阶动量校正 | `torch.optim.RAdam` | - 校正早期二阶动量方差，减少初始不稳定性<br>- 训练初期更稳定，接近AdamW性能<br>- 默认lr=0.001 | - CV、NLP任务<br>- 需要稳定初始训练的场景 |
| **Adamax** | 动量法 + 自适应学习率（无穷范数） | `torch.optim.Adamax` | - 使用梯度无穷范数代替二阶动量<br>- 对稀疏梯度或噪声大场景更稳定<br>- 默认lr=0.002，计算更简单 | - 稀疏数据（如NLP嵌入层）<br>- 噪声较大的优化任务 |
| **SparseAdam** | 动量法 + 自适应学习率（稀疏优化） | `torch.optim.SparseAdam` | - 专为稀疏梯度设计，仅更新非零梯度<br>- 节省计算和内存<br>- 不支持weight_decay | - 稀疏梯度场景（如nn.Embedding(sparse=True)）<br>- NLP中的词嵌入优化 |

### 数据集

#### Dataset示例
以下是一个简单的自定义Dataset示例，假设数据集包含图像和标签。

In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np

class SimpleDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        """
        初始化数据集
        Args:
            data: 特征数据（如图像或数值数组）
            labels: 对应的标签
            transform: 数据增强或预处理函数
        """
        self.data = data  # 假设为NumPy数组
        self.labels = labels
        self.transform = transform

    def __len__(self):
        """返回数据集大小"""
        return len(self.data)

    def __getitem__(self, index):
        """根据索引返回单个样本"""
        sample = self.data[index]
        label = self.labels[index]

        # 应用数据增强或预处理
        if self.transform:
            sample = self.transform(sample)

        # 转换为 PyTorch Tensor
        sample = torch.tensor(sample, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)
        return sample, label

#### 引用Dataset

In [ ]:
# 创建虚拟数据
data = np.random.randn(100, 3, 32, 32)  # 模拟生成100张3x32x32的图像
labels = np.random.randint(0, 10, size=100)  # 10个标签 (0-9)
dataset = SimpleDataset(data, labels)

# 访问数据集
sample, label = dataset[0]
print(f"Sample shape: {sample.shape}, Label: {label}")

#### 由DataLoader加载数据

**关键参数**
- batch_size：每批样本数，需根据GPU内存调整
- shuffle：训练时设为True，验证/测试时设为 False。
- num_workers：多线程加载，建议4-8（视CPU核心数）。
- pin_memory：启用以加速GPU数据传输。

In [ ]:
from torch.utils.data import DataLoader

# 创建 DataLoader
dataloader = DataLoader(
    dataset,
    batch_size=16,      # 批次大小
    shuffle=True,       # 打乱数据
    num_workers=4,      # 多线程加载
    #pin_memory=True     # 加速GPU传输
)

# 遍历数据
for batch_data, batch_labels in dataloader:
    print(f"Batch data shape: {batch_data.shape}, Batch labels shape: {batch_labels.shape}")
    # 送入模型训练...

#### 完整的极简示例

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# Step 1: 定义一个简单的数据集（继承 Dataset）
class MyDataset(Dataset):
    def __init__(self):
        # 随机生成 100 个样本，每个样本是 10 维特征向量
        self.data = torch.randn(100, 10)
        # 随机生成标签（0 或 1）
        self.labels = torch.randint(0, 2, (100,))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        return x, y

# Step 2: 实例化数据集
dataset = MyDataset()

# Step 3: 用 DataLoader 封装数据集
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Step 4: 模拟训练过程中读取数据
for batch_idx, (features, labels) in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}")
    print(f"  Features: {features.shape}")  # 应该是 [8, 10]
    print(f"  Labels:   {labels}")
    
    # 模拟一个训练步骤
    # output = model(features)
    # loss = loss_fn(output, labels)
    # loss.backward()
    # optimizer.step()

    if batch_idx == 2:  # 演示用：只打印前 3 个批次
        break

#### 使用自定义义collate_fn处理变长序列

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# --- 1. 定义 TextDataset (来自上一个教程) ---
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        # 假设文本已转换为ID序列的Tensor
        self.texts = [torch.tensor(t, dtype=torch.long) for t in texts]
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# --- 2. 定义自定义 collate_fn ---
def custom_collate_fn_for_padding(batch):
    """
    自定义 collate_fn 来处理变长序列填充。
    Args:
        batch (list): 包含 (text_tensor, label) 元组的列表。
                      例如：[(tensor([1,2,3]), 0), (tensor([4,5]), 1)]
    Returns:
        tuple: 包含填充后的文本批次和标签批次的元组。
    """
    texts, labels = zip(*batch) # 解包批次，得到文本列表和标签列表

    # 获取批次中最长的文本序列长度
    max_len = max([len(t) for t in texts])

    # 对所有文本序列进行填充，使其达到 max_len
    # 填充值通常选择0或其他特殊token_id
    padded_texts = []
    for t in texts:
        padded_text = torch.cat([t, torch.zeros(max_len - len(t), dtype=t.dtype)])
        padded_texts.append(padded_text)

    # 将填充后的文本序列堆叠成一个批次Tensor
    texts_batch = torch.stack(padded_texts)
    labels_batch = torch.tensor(labels, dtype=torch.long)

    return texts_batch, labels_batch

# --- 3. 准备模拟变长文本数据 ---
mock_texts = [
    [101, 200, 301, 405],
    [101, 502],
    [101, 603, 704, 805, 906, 107],
    [101]
] # 假设是词ID序列，101是CLS token
mock_text_labels = [0, 1, 0, 1]

# --- 4. 创建 Dataset 实例 ---
text_dataset = TextDataset(mock_texts, mock_text_labels)

# --- 5. 创建 DataLoader 实例，并传入 custom_collate_fn_for_padding ---
text_loader = DataLoader(
    dataset=text_dataset,
    batch_size=2,
    shuffle=False, # 为了方便观察填充效果，不洗牌
    collate_fn=custom_collate_fn_for_padding # 使用自定义的合并函数
)

# --- 6. 迭代 DataLoader 并观察输出 ---
print("\n--- Using DataLoader with custom collate_fn for variable-length sequences ---")
for batch_idx, (text_batch, label_batch) in enumerate(text_loader):
    print(f"\nBatch {batch_idx}:")
    print(f"  Text batch shape: {text_batch.shape}")
    print(f"  Text batch content:\n{text_batch}")
    print(f"  Label batch: {label_batch.tolist()}")

    # 预期输出：
    # Batch 0: (最大长度为 4)
    #   Text batch shape: torch.Size([2, 4])
    #   Text batch content:
    #   tensor([[101, 200, 301, 405],
    #           [101, 502,   0,   0]])
    #   Label batch: [0, 1]
    #
    # Batch 1: (最大长度为 6)
    #   Text batch shape: torch.Size([2, 6])
    #   Text batch content:
    #   tensor([[101, 603, 704, 805, 906, 107],
    #           [101,   0,   0,   0,   0,   0]])
    #   Label batch: [0, 1]

#### 使用torchvision.datasets示例

In [ ]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 定义数据增强
transform = transforms.Compose([
    transforms.ToTensor(),  # 转换为 Tensor
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST 均值和标准差
])

# 加载数据集
train_dataset = torchvision.datasets.MNIST(
    root='./data',      # 数据存储路径
    train=True,         # 训练集
    download=True,      # 自动下载
    transform=transform  # 数据增强
)

test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# 创建 DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# 遍历数据
for images, labels in train_loader:
    print(f"Images shape: {images.shape}, Labels shape: {labels.shape}")
    break

## MLP示例

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义 MLP 模型
class MLP(nn.Module):
    def __init__(self, input_size=784, hidden_size=256, output_size=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = x.view(-1, 784)  # 展平输入 (batch_size, 784)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)  # 输出层不加激活（由损失函数处理）
        return x

# 数据准备
transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 初始化模型、损失函数和优化器
model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练循环
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()  # 清空梯度
        outputs = model(images)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")
    
    # 测试
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

# 保存模型
torch.save(model.state_dict(), 'mlp_mnist.pth')

## 模型参数初始化

下面用PyTorch演示一个完整的“参数初始化”的代码流程，内容包括：
- 构建一个简单的神经网络模型（包含Linear和BatchNorm层）
- 查看默认的初始化结果
- 自定义初始化方法（如Xavier和Kaiming）
- 应用自定义初始化并验证效果

In [ ]:
# 第一步：导入库
import torch
import torch.nn as nn

# 设置随机种子以确保结果可重复
torch.manual_seed(42)

# 第二步：定义一个简单的MLP模型
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(10, 32)      # 输入层 -> 隐藏层
        self.bn1 = nn.BatchNorm1d(32)     # 批归一化
        self.relu = nn.ReLU()             # 激活函数
        self.fc2 = nn.Linear(32, 2)       # 隐藏层 -> 输出层（2类）

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 实例化模型
model = SimpleMLP()

# 第三步：查看默认初始化的参数分布
print("默认初始化参数（均值 & 标准差）:")
for name, param in model.named_parameters():
    #print(name, param)
    if 'weight' in name or 'bias' in name:
        print(f"{name:20s} | mean = {param.mean():.4f}, std = {param.std():.4f}")

# 第四步：定义自定义初始化函数
def custom_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)     # 使用 Xavier 均匀初始化权重
        # 说明：可用以下三种初始化方法替代上面的初始化方法，进行对比测试
        # nn.init.kaiming_normal_(m.weight, nonlinearity='relu')  # Kaiming初始化（适用于ReLU激活）
        # nn.init.normal_(m.weight, mean=0.0, std=0.02)  # 正态分布
        # nn.init.uniform_(m.weight, a=-0.1, b=0.1)   # 均匀分布
        nn.init.constant_(m.bias, 0)          # 偏置初始化为 0
    elif isinstance(m, nn.BatchNorm1d):
        nn.init.constant_(m.weight, 1)        # γ = 1
        nn.init.constant_(m.bias, 0)          # β = 0

# 应用自定义初始化
model.apply(custom_init)

# 第五步：查看初始化后的参数分布
print("\n自定义初始化参数（均值 & 标准差）:")
for name, param in model.named_parameters():
    if 'weight' in name or 'bias' in name:
        print(f"{name:20s} | mean = {param.mean():.4f}, std = {param.std():.4f}")

# 第六步（可选）：验证模型输出是否正常
# 构造一个随机输入batch（大小为4，特征维度为10）
x = torch.randn(4, 10)
output = model(x)

print("\n模型输出:")
print(output)

### 模型中需要进行初始化的位置

| 层类型                     | 需要初始化的参数                                                                 | 常见初始化方法                                                                 |
|----------------------------|----------------------------------------------------------------------------------|----------------------------------------------------------------------------------|
| 全连接层                   | 权重矩阵 \( W \)（形状：$[n_{\text{in}}, n_{\text{out}}]$）<br>偏置向量 \( b \)（形状：$[n_{\text{out}}]$） | 权重：Xavier（tanh/sigmoid）、He（ReLU）、正交初始化<br>偏置：零初始化或小常数（如0.01） |
| 卷积层                     | 卷积核（权重）（形状：$[k_h, k_w, c_{\text{in}}, c_{\text{out}}]$）<br>偏置向量（形状：$[c_{\text{out}}]$） | 权重：He（ReLU）、Xavier<br>偏置：零初始化                                       |
| 循环神经网络（RNN/LSTM/GRU） | 权重矩阵（如 $ W_{xh}, W_{hh} $，形状依层设计）<br>偏置向量（形状：$[n_{\text{hidden}}]$） | 权重：正交初始化、Xavier<br>偏置：零初始化，LSTM遗忘门偏置可为1                |
| 注意力机制（Attention）      | 权重矩阵（如 $ W_Q, W_K, W_V $，形状：$[d_{\text{model}}, d_k/d_v]$）<br>偏置向量（部分实现） | 权重：Xavier、He<br>偏置：零初始化                                              |
| 嵌入层（Embedding）         | 嵌入矩阵（形状：$[n_{\text{vocab}}, d_{\text{emb}}]$）                          | 随机初始化（均匀/正态分布）、预训练嵌入（如GloVe）、正弦/余弦位置编码            |
| 批归一化（Batch Norm）      | 缩放参数 $ \gamma $（通常为1）<br>偏移参数 $ \beta $（通常为0）                | $ \gamma \sim 1 $，$ \beta \sim 0 $                                         |

**备注**：
- 池化层、激活函数、Dropout层通常无需初始化参数。
- 初始化方法需根据激活函数（如ReLU、tanh）和网络深度选择。

## 正则化技术

| 正则化技术 | 定义 | 作用 | 优点 | 缺点 | 应用场景 |
|------------|------|------|------|------|----------|
| **L1正则化** | 在损失函数中加入参数的L1范数（绝对值和） | 产生稀疏解，实现特征选择，降低模型复杂度 | 1. 稀疏模型便于解释<br>2. 适合高维数据 | 1. 非光滑优化问题<br>2. 对噪声敏感 | 线性模型（如Lasso回归）、稀疏神经网络 |
| **L2正则化** | 在损失函数中加入参数的L2范数（平方和） | 平滑权重，降低对噪声的敏感性 | 1. 优化光滑，便于梯度下降<br>2. 增强模型稳定性 | 1. 不产生稀疏解<br>2. 需调整正则化强度 | 线性回归、逻辑回归、神经网络（权重衰减） |
| **Dropout** | 训练时随机丢弃部分神经元 | 防止过度依赖某些神经元，模拟模型集成 | 1. 简单高效<br>2. 减少过拟合 | 1. 增加训练时间<br>2. 测试时需权重调整 | 深度神经网络（如CNN、RNN） |
| **数据增强** | 对训练数据进行变换（如旋转、翻转） | 增加数据多样性，提高泛化能力 | 1. 不改变模型结构<br>2. 适合视觉/语音任务 | 1. 需设计合适策略<br>2. 增加预处理成本 | 图像分类、语音处理、NLP |
| **早停** | 验证集误差不再下降时停止训练 | 限制训练轮数，防止过拟合 | 1. 简单易实现<br>2. 无需改模型 | 1. 需划分验证集<br>2. 可能导致欠拟合 | 深度学习、梯度提升树 |
| **批标准化** | 规范化每层输入的均值和方差 | 减少内部协变量偏移，稳定训练 | 1. 加速收敛<br>2. 减少对Dropout依赖 | 1. 小批量效果差<br>2. 增加计算开销 | 深度神经网络（CNN、全连接网络） |
| **弹性网正则化** | 结合L1和L2正则化 | 综合稀疏性和平滑性 | 1. 兼具特征选择和平滑<br>2. 适合复杂特征相关性 | 1. 需调两个超参数<br>2. 增加调参难度 | 高维特征回归、稀疏线性模型 |
| **噪声注入** | 在数据/权重中添加随机噪声 | 增强模型鲁棒性 | 1. 简单易实现<br>2. 提高抗噪能力 | 1. 需调整噪声水平<br>2. 可能破坏数据结构 | 语音识别、图像处理、时间序列 |
| **参数共享** | 强制模型共享部分参数（如卷积核） | 减少自由参数，降低复杂度 | 1. 降低计算量<br>2. 适合结构化数据 | 1. 仅限特定模型<br>2. 适用范围有限 | 卷积神经网络、循环神经网络 |

### L1和L2正则化示例

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义简单模型
model = nn.Linear(10, 1)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)  # L2 正则化（权重衰减）

# 训练数据
x = torch.randn(32, 10)
y = torch.randn(32, 1)

# 前向传播
output = model(x)
loss = criterion(output, y)

# 手动添加 L1 正则化
l1_lambda = 0.01
l1_norm = sum(p.abs().sum() for p in model.parameters())  # 计算L1范数
loss += l1_lambda * l1_norm

# 反向传播和优化
optimizer.zero_grad()
loss.backward()
optimizer.step()

### Dropout示例

In [ ]:
import torch
import torch.nn as nn

# 定义带 Dropout 的模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10, 20)
        self.dropout = nn.Dropout(p=0.5)  # 50% 概率丢弃
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # 应用 Dropout
        x = self.fc2(x)
        return x

# 示例
model = Net()
model.train()  # 启用 Dropout
x = torch.randn(32, 10)
output = model(x)

### Batch Normalization示例

In [ ]:
import torch
import torch.nn as nn

# 定义带批标准化的模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(16)  # 批标准化，16个通道
        self.relu = nn.ReLU()
        self.fc = nn.Linear(16 * 32 * 32, 10)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)  # 应用批标准化
        x = self.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 示例
model = Net()
model.train()  # 启用批标准化
x = torch.randn(32, 3, 32, 32)
output = model(x)

### 早停法示例

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义模型和数据
model = nn.Linear(10, 1)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
train_loader = ...  # 假设已有训练和验证数据加载器
val_loader = ...

# 提前停止参数
patience = 10
best_val_loss = float('inf')
counter = 0
best_model_state = None

# 训练循环
for epoch in range(100):
    model.train()
    for x, y in train_loader:
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

    # 验证
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x, y in val_loader:
            output = model(x)
            val_loss += criterion(output, y).item()
    val_loss /= len(val_loader)

    # 提前停止逻辑
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

# 加载最佳模型
model.load_state_dict(best_model_state)

## 模型保存与加载

### 查看state_dict

In [ ]:
import torch
import torch.nn as nn

# 定义一个简单的前馈神经网络 (FNN)
class SimpleFNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # 全连接层 1
        self.relu = nn.ReLU()                         # 激活函数 (没有参数)
        self.fc2 = nn.Linear(hidden_size, output_size) # 全连接层 2

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 实例化模型
input_size = 10
hidden_size = 20
output_size = 3
model = SimpleFNN(input_size, hidden_size, output_size)

# 获取模型的 state_dict
model_state_dict = model.state_dict()

# 打印 state_dict 的类型
print(f"State dict type: {type(model_state_dict)}")

# 打印 state_dict 中的键和对应的张量形状
print("\nKeys and shapes in state_dict:")
for key, value in model_state_dict.items():
    print(f"Key: {key}, Shape: {value.shape}")

# 打印其中一个参数的张量内容 (仅部分展示)
print(f"\nContent of '{list(model_state_dict.keys())[0]}':")
print(model_state_dict[list(model_state_dict.keys())[0]])

### 查看持续缓冲区

给上面的示例添加一个BatchNorm层，state_dict字典中在可学习参数之外，还会多出持久缓冲区。

下面示例中的bn1层不仅有可学习参数weight和bias（对应BatchNorm的$\gamma$和$\beta$参数，它们是可学习的），还有三个缓冲区：running_mean、running_var和num_batches_tracked。这些缓冲区是模型状态的重要组成部分，在推理时需要使用，因此也包含在state_dict中。

In [ ]:
import torch
import torch.nn as nn

class FNNWithBatchNorm(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNNWithBatchNorm, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        # BatchNorm 1D 常用于全连接层之后
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x) # 应用 BatchNorm
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 实例化模型
input_size = 10
hidden_size = 20
output_size = 3
model_bn = FNNWithBatchNorm(input_size, hidden_size, output_size)

# 获取模型的 state_dict
model_bn_state_dict = model_bn.state_dict()

# 打印 state_dict 中的键和对应的张量形状
print("\nKeys and shapes in state_dict (with BatchNorm):")
for key, value in model_bn_state_dict.items():
    print(f"Key: {key}, Shape: {value.shape}")

### 保存和加载state_dict的完整的示例

#### 保存state_dict

In [ ]:
import torch
import torch.nn as nn
import os # 用于检查文件是否存在

# 假设我们已经定义并（可能）训练了一个 SimpleFNN 模型实例
# （这里我们为了演示，只实例化，不进行实际训练）

class SimpleFNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 实例化模型
input_size = 10
hidden_size = 20
output_size = 3
model = SimpleFNN(input_size, hidden_size, output_size)

# 定义保存文件的路径
model_params_path = 'simple_fnn_params.pth' # 常用的文件扩展名是 .pt 或 .pth

# --- 保存模型的 state_dict ---
print(f"Saving model state_dict to {model_params_path}...")
try:
    # 获取模型的 state_dict
    state_dict_to_save = model.state_dict()

    # 使用 torch.save() 保存 state_dict
    torch.save(state_dict_to_save, model_params_path)

    print("Model state_dict saved successfully!")
    print(f"Checking if file exists: {os.path.exists(model_params_path)}")

except Exception as e:
    print(f"An error occurred while saving: {e}")

#### 加载state_dict

In [ ]:
# --- 加载 state_dict ---
print(f"\nLoading state_dict from {model_params_path}...")
try:
    # 使用 torch.load() 加载 state_dict
    loaded_state_dict = torch.load(model_params_path)

    print("State dict loaded successfully!")
    print(f"Type of loaded object: {type(loaded_state_dict)}")

    # 我们可以像查看原始 state_dict 一样查看加载的字典
    print("\nKeys in loaded state_dict:")
    for key in loaded_state_dict.keys():
        print(key)

except FileNotFoundError:
    print(f"Error: The file '{model_params_path}' was not found.")
except Exception as e:
    print(f"An error occurred while loading: {e}")

# --- 将加载的 state_dict 载入到新的模型实例 ---
print("\nLoading state_dict into a new model instance...")

# !!! 关键步骤 1: 创建一个新的模型实例 !!!
# 这个实例的结构 (层、层的大小、层之间的连接) 必须与保存 state_dict 时使用的模型一致
new_model = SimpleFNN(input_size, hidden_size, output_size)
print("New model instance created.")

try:
    # !!! 关键步骤 2: 将加载的state_dict载入到新模型实例中 !!!
    # load_state_dict 将字典中的参数复制到模型的对应位置
    # 默认情况下 (strict=True), 加载的state_dict的键必须精确匹配模型实例的state_dict
    new_model.load_state_dict(loaded_state_dict)

    print("State dict loaded into new model instance successfully!")

    # 加载完成后，新模型的参数就和保存时的模型参数完全一样了

except RuntimeError as e:
    # 如果模型结构不匹配，load_state_dict会抛出RuntimeError
    print(f"Error loading state dict: Model structure mismatch. Details:\n{e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

#### 推理模式的特殊设定

In [ ]:
# 继续上面的例子...

# 将加载参数后的模型设置为评估模式
new_model.eval()
print("\nModel set to evaluation mode.")

# 模拟一个输入张量 (通常是一个 batch 的数据)
dummy_input = torch.randn(1, input_size) # batch size = 1

# 在 no_grad 上下文下进行推理
print("Performing inference with the loaded model...")
with torch.no_grad():
    output = new_model(dummy_input)

print("Inference successful.")
print(f"Output shape: {output.shape}")
# 模型的输出现在是基于加载的参数计算得到的。



#### state_dict的严格模式与非严格模式

In [ ]:
import torch
import torch.nn as nn

# 定义 FNN 版本 1
class SimpleFNN_v1(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleFNN_v1, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 定义 FNN 版本 2 (比版本 1 多一层)
class SimpleFNN_v2(nn.Module):
    def __init__(self, input_size, hidden_size, intermediate_size, output_size):
        super(SimpleFNN_v2, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        # 注意这里的输出层大小与SimpleFNN_v1有所差别，即使stirct=False，
        # 仍会导致加载失败，因为非严格模式并不能接受形状上的不兼容，所以需要加载前进行特殊处理。
        self.fc2 = nn.Linear(hidden_size, intermediate_size) 
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(intermediate_size, output_size) # 新增一层
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# 模型参数
input_size = 10
hidden_size = 20
intermediate_size = 15
output_size = 3

# 实例化并保存 v1 模型的 state_dict
model_v1 = SimpleFNN_v1(input_size, hidden_size, output_size)
v1_state_dict = model_v1.state_dict()

# 实例化 v2 模型
model_v2 = SimpleFNN_v2(input_size, hidden_size, intermediate_size, output_size)

print("Loading V1 state_dict into V2 model with strict=False and checking return info...")
# 获取目标模型 (v2) 的 state_dict，用于检查形状
v2_model_dict = model_v2.state_dict()

# 创建一个新的 state_dict，只包含 v1 中存在且在 v2 中形状匹配的键值对
# 这是解决 size mismatch 问题的关键
filtered_state_dict = {k: v for k, v in v1_state_dict.items() if
                       k in v2_model_dict and v.shape == v2_model_dict[k].shape}

print("\nFiltered state_dict keys to load (matching shapes):")
if filtered_state_dict:
    for k in filtered_state_dict.keys():
        print(f"  {k}")
else:
    print("  No keys with matching names and shapes found.")

# 使用 strict=False 加载过滤后的 state_dict，此时不会再有 size mismatch，因为我们只加载形状匹配的层。
# strict=False 会忽略 v2 中存在但 filtered_state_dict 中不存在的键 (fc2, fc3)。
try:
    load_info = model_v2.load_state_dict(filtered_state_dict, strict=False)

    print("\nLoading completed successfully.")
    # load_info 会包含加载过程中被忽略的键 (missing_keys) 和 state_dict 中存在但在模型中不存在的键 (unexpected_keys)。
    # 在这里，fc2 和 fc3 会是 missing_keys，因为它们在 model_v2 中，但不在 filtered_state_dict 中。
    print(f"Missing keys (exist in model_v2 but not in filtered_state_dict): {load_info.missing_keys}")
    # unexpected_keys 应该为空，因为我们已经过滤掉了所有不在 model_v2 中的键
    print(f"Unexpected keys (exist in filtered_state_dict but not in model_v2): {load_info.unexpected_keys}")

except Exception as e:
    # 现在应该不会因为 size mismatch 而进入这个 except 块了
    print(f"An unexpected error occurred during loading: {e}")
    # 在异常发生时，load_info 可能未定义，这里为了安全，可以捕获一下或者在del前检查
    load_info = None # 确保即使出错 load_info 也有一个值，避免 NameError

# 清理
del load_info, model_v1, model_v2, v1_state_dict, filtered_state_dict, v2_model_dict
print("\nCleanup complete.")

### 保存和加载整个模型

#### 模型整个模型

In [ ]:
import torch
import torch.nn as nn
import os # 用于检查文件是否存在

# 假设我们已经定义并实例化了一个 SimpleFNN 模型
# （这里我们为了演示，只实例化）

class SimpleFNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 定义模型参数
input_size = 10
hidden_size = 20
output_size = 3

# 实例化要保存的模型
model_to_save = SimpleFNN(input_size, hidden_size, output_size)

# 定义保存整个模型的文件路径
entire_model_path = 'entire_simple_fnn_model.pth' # 不同的文件名以示区别

# --- 保存整个模型实例 ---
print(f"Saving entire model instance to {entire_model_path}...")
try:
    # 使用 torch.save() 保存整个模型对象
    torch.save(model_to_save, entire_model_path)

    print("Entire model instance saved successfully!")
    print(f"Checking if file exists: {os.path.exists(entire_model_path)}")

except Exception as e:
    print(f"An error occurred while saving: {e}")

#### 加载整个模型

In [ ]:
# --- 加载整个模型实例 ---
print(f"\nLoading entire model instance from {entire_model_path}...")

try:
    # 使用torch.load()直接加载整个模型对象
    # 注意：Pytorch 2.6版本之前，weights_only参数的默认值为False,但该版本之后的默认值为True，
    # 因此，若要加载整个模型，需要手动将其设置为False。
    loaded_model_instance = torch.load(entire_model_path, weights_only=False)

    print("Entire model instance loaded successfully!")
    print(f"Type of loaded object: {type(loaded_model_instance)}")

    # 您可以直接使用加载后的模型实例进行推理
    print("\nLoaded model instance created.")

    # 加载完成后，模型已经包含结构和参数

    # 类似 state_dict 方法，加载用于推理的模型通常需要设置为评估模式
    loaded_model_instance.eval()
    print("Loaded model set to evaluation mode.")

    # 模拟输入进行推理
    dummy_input = torch.randn(1, input_size)
    with torch.no_grad():
        output = loaded_model_instance(dummy_input)

    print("Inference successful with loaded entire model.")
    print(f"Inference output shape: {output.shape}")


except FileNotFoundError:
    print(f"Error: The file '{entire_model_path}' was not found.")
except Exception as e:
    # !!! 注意：如果模型类定义不可用或不匹配，这里会抛出各种错误 !!!
    print(f"An error occurred while loading or inferring: {e}")

### 保存和加载检查点文件示例

#### 构建检查点字典

In [ ]:
# 假设我们有以下对象和变量
# model: 当前训练中的模型实例
# optimizer: 当前训练中的优化器实例
# epoch: 当前完成的 epoch 数
# step: 当前完成的总步数 (可选)
# loss: 当前的训练 loss (可选)
# best_metric: 历史最佳评估指标 (可选)
# scheduler: 当前使用的学习率调度器实例 (如果使用的话)

import torch
import torch.nn as nn
import torch.optim as optim

# 示例：假设我们已经定义了模型、优化器，并且进行了一段训练
class SimpleFNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 实例化模型和优化器 (这里只是为了构建 state_dict，不进行实际训练)
input_size = 10
hidden_size = 20
output_size = 3
lr=0.001
model = SimpleFNN(input_size, hidden_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=lr)

# 模拟一些训练状态
current_epoch = 5
current_step = 500
current_loss = 0.15
best_validation_accuracy = 0.88

# 如果使用了学习率调度器，也获取其状态字典
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
# scheduler_state = scheduler.state_dict() # 需要先定义和初始化 scheduler

# --- 构建检查点字典 ---
checkpoint = {
    'epoch': current_epoch,
    'step': current_step,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': current_loss,
    'best_accuracy': best_validation_accuracy,
    # 'scheduler_state_dict': scheduler_state, # 如果使用调度器则包含此项
    # 'torch_rng_state': torch.get_rng_state(), # 如果需要严格复现，保存 RNG 状态
    # 'numpy_rng_state': np.random.get_state(), # 如果使用 NumPy，保存 RNG 状态
    # 'random_rng_state': random.getstate(),  # 如果使用 Python random，保存 RNG 状态
}

print("Checkpoint dictionary built:")
print(checkpoint.keys())
# 打印其中一个 state_dict 的键来确认
print("\nKeys in model_state_dict within checkpoint:")
print(checkpoint['model_state_dict'].keys())

# 注意：如果模型和优化器没有进行过实际的训练步骤，它们的 state_dict 可能不完整（特别是优化器）
# 在实际训练中，确保在保存检查点时，模型和优化器已经进行了至少一步更新。

#### 保存检查点文件

In [ ]:
# 继续上面的例子...

# 定义检查点文件保存路径
checkpoint_path = 'my_training_checkpoint.pth' # 或者根据 epoch/性能命名

# --- 保存检查点字典 ---
print(f"\n保存检查点信息至文件 {checkpoint_path} 中...")
try:
    torch.save(checkpoint, checkpoint_path)
    print("检查点文件保存成功!")
    print(f"检查文件是否存在: {os.path.exists(checkpoint_path)}")

except Exception as e:
    print(f"An error occurred while saving checkpoint: {e}")

# 清理
# del model, optimizer, checkpoint # 在实际脚本中，这些通常会一直存在

#### 从检查点恢复模型

In [ ]:
# 继续上面的例子...

# 定义检查点文件路径 (假设它已经存在)
checkpoint_path = 'my_training_checkpoint.pth'

# --- 加载检查点以恢复训练 ---
print(f"\n正在从检查点文件 {checkpoint_path} 开始恢复训练...")

# 确定加载设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Loading onto device: {device}")

try:
    # 1. 加载检查点字典，并指定映射设备
    checkpoint = torch.load(checkpoint_path, map_location=device)
    print("检查点字典装载完成...")

    # 2. 实例化模型 (结构必须与保存时一致)
    loaded_model = SimpleFNN(input_size, hidden_size, output_size)
    # 将模型移动到目标设备
    loaded_model.to(device)
    print("创建新模型并移动到指定的设备...")

    # 3. 实例化优化器 (类型和配置应与保存时一致)
    # !!! 使用加载参数后的模型的参数初始化优化器 !!!
    loaded_optimizer = optim.Adam(loaded_model.parameters(), lr=lr) # 初始 lr 会被 state_dict 覆盖
    print("创建新的优化器实例...")

    # 4. 加载模型状态
    loaded_model.load_state_dict(checkpoint['model_state_dict'])
    print("从检查点文件加载模型的state_dict...")

    # 5. 加载优化器状态
    loaded_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    print("从检查点文件加载优化器的state_dict...")

    # 6. 恢复其他状态
    start_epoch = checkpoint['epoch'] + 1 # 从下一个 epoch 开始
    recovered_step = checkpoint['step']
    recovered_loss = checkpoint['loss']
    recovered_best_accuracy = checkpoint['best_accuracy']

    # 如果使用了调度器
    # loaded_scheduler = optim.lr_scheduler.StepLR(loaded_optimizer, step_size=30, gamma=0.1)
    # loaded_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    # print("Scheduler state loaded.")

    # 如果保存了 RNG 状态并需要严格复现
    # torch.set_rng_state(checkpoint['torch_rng_state'])
    # np.random.set_state(checkpoint['numpy_rng_state'])
    # random.setstate(checkpoint['random_rng_state'])
    # print("RNG states restored.")

    print(f"\n从 Epoch {start_epoch}和 Step {recovered_step} 继续进行训练...")
    print(f"Recovered Loss: {recovered_loss}, Best Accuracy: {recovered_best_accuracy}")


    # 7. 设置模型模式 (恢复训练则设置为 train())
    loaded_model.train() # 设置为训练模式
    print("将模型设置为训练模式...")

except FileNotFoundError:
    print(f"Error: Checkpoint file not found at {checkpoint_path}. Starting training from scratch.")
    # 在实际应用中，如果文件不存在，你会选择从头开始训练
    # start_epoch = 0
    # loaded_model = SimpleFNN(...)
    # loaded_optimizer = optim.Adam(...)
    # etc.
except Exception as e:
     print(f"An unexpected error occurred during loading checkpoint: {e}")
     # 根据需要处理加载失败的情况
finally:
     # 清理生成的文件 (可选)
    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)
        print(f"\n清理检查点文件：{checkpoint_path}")